<a href="https://colab.research.google.com/github/d3ttl4ff/aceTennis/blob/main/steam_data_cleaning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Import Libraries**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Import the required libraries
import csv
import json
import os
import statistics
import time
import datetime
import sys
import numpy as np
import pandas as pd
from tabulate import tabulate
from ast import literal_eval
import itertools
import re
import ast
from bs4 import BeautifulSoup
from collections import Counter

# customisations
pd.set_option("display.max_columns", 100)

In [ ]:
# Define the paths
dataset_dir = "/content/drive/MyDrive/Colab Notebooks/steamdata/experimental/datasets"
dataset_final_dir = "/content/drive/MyDrive/Colab Notebooks/steamdata/experimental/datasets_final"
data_source_dir = "/content/drive/MyDrive/Colab Notebooks/steamdata/experimental/data"

In [ ]:
steam_dataset = pd.read_csv(f'{dataset_final_dir}/steam_dataset_final_ready_to_clean_4.csv')

<ipython-input-4-723a47d313f9>:1: DtypeWarning: Columns (22) have mixed types. Specify dtype option on import or set low_memory=False.
  steam_dataset = pd.read_csv(f'{dataset_final_dir}/steam_dataset_final_ready_to_clean_4.csv')


# **Inspect the Data**

In [ ]:
# print out number of rows and columns
print('Rows:', steam_dataset.shape[0])
print('Columns:', steam_dataset.shape[1])

# view first five rows
steam_dataset.head()

Rows: 93841
Columns: 42


,steamid,id,name_x,price,reviews_x,followers,reviewScore,copiesSold,revenue,wishlists,avgPlaytime,tags,genres_x,features,developers_x,publishers_x,unreleased,earlyAccess,releaseDate,publisherClass,players,owners,required_age,is_free,detailed_description,about_the_game,short_description,supported_languages,capsule_imagev5,pc_requirements,mac_requirements,linux_requirements,price_overview,platforms,categories,genres_y,achievements,demos,dlc,recommendations,metacritic,controller_support
0,10,10,Counter-Strike,9.99,247199,215044,97,7021860,29305366.0,277200.0,183.686513,"['Action', 'FPS', 'Multiplayer', 'Shooter', 'C...",['Action'],"['Online PvP', 'Shared/Split Screen PvP', 'Fam...",['Valve'],['Valve'],False,False,973054800000,AAA,11010395,16629587,0,False,Play the world's number 1 online action game. ...,Play the world's number 1 online action game. ...,Play the world's number 1 online action game. ...,"English<strong>*</strong>, French<strong>*</st...",https://shared.akamai.steamstatic.com/store_it...,{'minimum': '\n\t\t\t<p><strong>Minimum:</stro...,{'minimum': 'Minimum: OS X Snow Leopard 10.6....,"{'minimum': 'Minimum: Linux Ubuntu 12.04, Dual...","{'currency': 'USD', 'initial': 999, 'final': 9...","{'windows': True, 'mac': True, 'linux': True}","[{'id': 1, 'description': 'Multi-player'}, {'i...","[{'id': '1', 'description': 'Action'}]",NaN,NaN,NaN,{'total': 157796},"{'score': 88, 'url': 'https://www.metacritic.c...",NaN
1,20,20,Team Fortress Classic,4.99,8607,9617,87,375765,853393.0,37400.0,4.589017,"['Action', 'FPS', 'Multiplayer', 'Classic', 'H...",['Action'],"['Online PvP', 'Shared/Split Screen PvP', 'Rem...",['Valve'],['Valve'],False,False,922942800000,AAA,1415339,7823635,0,False,One of the most popular online action games of...,One of the most popular online action games of...,One of the most popular online action games of...,"English, French, German, Italian, Spanish - Sp...",https://shared.akamai.steamstatic.com/store_it...,{'minimum': '\n\t\t\t<p><strong>Minimum:</stro...,{'minimum': 'Minimum: OS X Snow Leopard 10.6....,"{'minimum': 'Minimum: Linux Ubuntu 12.04, Dual...","{'currency': 'USD', 'initial': 499, 'final': 4...","{'windows': True, 'mac': True, 'linux': True}","[{'id': 1, 'description': 'Multi-player'}, {'i...","[{'id': '1', 'description': 'Action'}]",NaN,NaN,NaN,{'total': 6442},NaN,NaN
2,30,30,Day of Defeat,4.99,7021,7244,90,230695,495765.0,37400.0,18.550989,"['FPS', 'World War II', 'Multiplayer', 'Shoote...",['Action'],['Family Sharing'],['Valve'],['Valve'],False,False,1051761600000,AAA,1492839,6938005,0,False,Enlist in an intense brand of Axis vs. Allied ...,Enlist in an intense brand of Axis vs. Allied ...,Enlist in an intense brand of Axis vs. Allied ...,"English, French, German, Italian, Spanish - Spain",https://shared.akamai.steamstatic.com/store_it...,{'minimum': '\n\t\t\t<p><strong>Minimum:</stro...,{'minimum': 'Minimum: OS X Snow Leopard 10.6....,"{'minimum': 'Minimum: Linux Ubuntu 12.04, Dual...","{'currency': 'USD', 'initial': 499, 'final': 4...","{'windows': True, 'mac': True, 'linux': True}","[{'id': 1, 'description': 'Multi-player'}, {'i...","[{'id': '1', 'description': 'Action'}]",NaN,NaN,NaN,{'total': 4218},"{'score': 79, 'url': 'https://www.metacritic.c...",NaN
3,40,40,Deathmatch Classic,4.99,3136,2723,83,143370,310525.0,14300.0,2.748242,"['Action', 'FPS', 'Classic', 'Multiplayer', 'S...",['Action'],"['Online PvP', 'Shared/Split Screen PvP', 'Rem...",['Valve'],['Valve'],False,False,991368000000,AAA,1010152,7236021,0,False,Enjoy fast-paced multiplayer gaming with Death...,Enjoy fast-paced multiplayer gaming with Death...,Enjoy fast-paced multiplayer gaming with Death...,"English, French, German, Italian, Spanish - Sp...",https://shared.akamai.steamstatic.com/store_it...,{'minimum': '\n\t\t\t<p><strong>Minimum:</stro...,{'minimum': 'Minimum: OS X Snow Leopard 10.6....,"{'minimum': 'Minimum: Linux Ubuntu 12.04, Dual...","{'currency': 'USD', 'initial': 499, 'final': 4...","{'windows': True, 'mac

In [ ]:
null_counts = steam_dataset.isnull().sum()
null_counts

,0
steamid,0
id,0
name_x,0
price,0
reviews_x,0
followers,0
reviewScore,0
copiesSold,0
revenue,0
wishlists,73


In [ ]:
threshold = steam_dataset.shape[0] // 2

print('Drop columns with more than {} missing rows'.format(threshold))
print()

drop_rows = steam_dataset.columns[null_counts > threshold]

print('Columns to drop: {}'.format(list(drop_rows)))

Drop columns with more than 46920 missing rows

Columns to drop: ['demos', 'dlc', 'recommendations', 'metacritic', 'controller_support']


In [ ]:
# Check for the unique values in every column
unique_counts = pd.DataFrame.from_records([(col, steam_dataset[col].nunique()) for col in steam_dataset.columns],
                                          columns=['Column_Name', 'Num_Unique'])
print(unique_counts)

             Column_Name  Num_Unique
0                steamid       93841
1                     id       93841
2                 name_x       93116
3                  price         345
4              reviews_x        5973
5              followers       11197
6            reviewScore          98
7             copiesSold       22167
8                revenue       30083
9              wishlists        1581
10           avgPlaytime       82812
11                  tags       87705
12              genres_x        2013
13              features        5842
14          developers_x       58090
15          publishers_x       50946
16            unreleased           1
17           earlyAccess           2
18           releaseDate        5568
19        publisherClass           4
20               players       25887
21                owners       29084
22          required_age          28
23               is_free           2
24  detailed_description       93478
25        about_the_game       93475
2

# **Drop Initial Columns**

In [ ]:
steam_dataset['unreleased'].value_counts(dropna=False)

,count
unreleased,
False,93841


In [ ]:
steam_dataset['earlyAccess'].value_counts(dropna=False)

,count
earlyAccess,
False,84453
True,9388


In [ ]:
def drop_null_cols(df, thresh=0.5):
    """Drop columns with more than a certain proportion of missing values (Default 50%)."""
    cutoff_count = len(df) * thresh

    return df.dropna(thresh=cutoff_count, axis=1)

def process(df):
    """Process data set. Will eventually contain calls to all functions we write."""

    # Copy the input dataframe to avoid accidentally modifying original data
    df = df.copy()

    # Remove duplicate rows - all appids should be unique
    df = df.drop_duplicates()

    # Remove columns with more than 50% null values
    df = drop_null_cols(df)

    # Remove rows where 'unreleased' is True or 'earlyAccess' is True
    df = df[(df['unreleased'] == False) & (df['earlyAccess'] == False)]

    # Drop the 'unreleased' and 'earlyAccess' columns after filtering
    df = df.drop(columns=['unreleased', 'earlyAccess'])

    return df

print(steam_dataset.shape)
initial_processing = process(steam_dataset)
print(initial_processing.shape)
initial_processing.head()

(93841, 42)
(84453, 35)


,steamid,id,name_x,price,reviews_x,followers,reviewScore,copiesSold,revenue,wishlists,avgPlaytime,tags,genres_x,features,developers_x,publishers_x,releaseDate,publisherClass,players,owners,required_age,is_free,detailed_description,about_the_game,short_description,supported_languages,capsule_imagev5,pc_requirements,mac_requirements,linux_requirements,price_overview,platforms,categories,genres_y,achievements
0,10,10,Counter-Strike,9.99,247199,215044,97,7021860,29305366.0,277200.0,183.686513,"['Action', 'FPS', 'Multiplayer', 'Shooter', 'C...",['Action'],"['Online PvP', 'Shared/Split Screen PvP', 'Fam...",['Valve'],['Valve'],973054800000,AAA,11010395,16629587,0,False,Play the world's number 1 online action game. ...,Play the world's number 1 online action game. ...,Play the world's number 1 online action game. ...,"English<strong>*</strong>, French<strong>*</st...",https://shared.akamai.steamstatic.com/store_it...,{'minimum': '\n\t\t\t<p><strong>Minimum:</stro...,{'minimum': 'Minimum: OS X Snow Leopard 10.6....,"{'minimum': 'Minimum: Linux Ubuntu 12.04, Dual...","{'currency': 'USD', 'initial': 999, 'final': 9...","{'windows': True, 'mac': True, 'linux': True}","[{'id': 1, 'description': 'Multi-player'}, {'i...","[{'id': '1', 'description': 'Action'}]",NaN
1,20,20,Team Fortress Classic,4.99,8607,9617,87,375765,853393.0,37400.0,4.589017,"['Action', 'FPS', 'Multiplayer', 'Classic', 'H...",['Action'],"['Online PvP', 'Shared/Split Screen PvP', 'Rem...",['Valve'],['Valve'],922942800000,AAA,1415339,7823635,0,False,One of the most popular online action games of...,One of the most popular online action games of...,One of the most popular online action games of...,"English, French, German, Italian, Spanish - Sp...",https://shared.akamai.steamstatic.com/store_it...,{'minimum': '\n\t\t\t<p><strong>Minimum:</stro...,{'minimum': 'Minimum: OS X Snow Leopard 10.6....,"{'minimum': 'Minimum: Linux Ubuntu 12.04, Dual...","{'currency': 'USD', 'initial': 499, 'final': 4...","{'windows': True, 'mac': True, 'linux': True}","[{'id': 1, 'description': 'Multi-player'}, {'i...","[{'id': '1', 'description': 'Action'}]",NaN
2,30,30,Day of Defeat,4.99,7021,7244,90,230695,495765.0,37400.0,18.550989,"['FPS', 'World War II', 'Multiplayer', 'Shoote...",['Action'],['Family Sharing'],['Valve'],['Valve'],1051761600000,AAA,1492839,6938005,0,False,Enlist in an intense brand of Axis vs. Allied ...,Enlist in an intense brand of Axis vs. Allied ...,Enlist in an intense brand of Axis vs. Allied ...,"English, French, German, Italian, Spanish - Spain",https://shared.akamai.steamstatic.com/store_it...,{'minimum': '\n\t\t\t<p><strong>Minimum:</stro...,{'minimum': 'Minimum: OS X Snow Leopard 10.6....,"{'minimum': 'Minimum: Linux Ubuntu 12.04, Dual...","{'currency': 'USD', 'initial': 499, 'final': 4...","{'windows': True, 'mac': True, 'linux': True}","[{'id': 1, 'description': 'Multi-player'}, {'i...","[{'id': '1', 'description': 'Action'}]",NaN
3,40,40,Deathmatch Classic,4.99,3136,2723,83,143370,310525.0,14300.0,2.748242,"['Action', 'FPS', 'Classic', 'Multiplayer', 'S...",['Action'],"['Online PvP', 'Shared/Split Screen PvP', 'Rem...",['Valve'],['Valve'],991368000000,AAA,1010152,7236021,0,False,Enjoy fast-paced multiplayer gaming with Death...,Enjoy fast-paced multiplayer gaming with Death...,Enjoy fast-paced multiplayer gaming with Death...,"English, French, German, Italian, Spanish - Sp...",https://shared.akamai.steamstatic.com/store_it...,{'minimum': '\n\t\t\t<p><strong>Minimum:</stro...,{'minimum': 'Minimum: OS X Snow Leopard 10.6....,"{'minimum': 'Minimum: Linux Ubuntu 12.04, Dual...","{'currency': 'USD', 'initial': 499, 'final': 4...","{'windows': True, 'mac': True, 'linux': True}","[{'id': 1, 'description': 'Multi-player'}, {'i...","[{'id': '1', 'description': 'Action'}]",NaN
4,50,50,Half-Life: Opposing Force,4.99,24809,10844,95,769170,1571654.0,51700.0,6.825604,"['FPS', 'Action', 'Classic', 'Sci-fi', 'Single...",['Action'],"['Single-player', 'Family Sharing']",['Gearbox 

In [ ]:
def drop_unwanted_columns(df):
    columns_to_drop = [
        "Unnamed: 0",
        "type",
        "header_image",
        "capsule_image",
        "capsule_imagev5",
        "website",
        "screenshots",
        "movies",
        "support_info",
        "background",
        "background_raw",
        "ext_user_account_notice",
        "drm_notice",
        "legal_notice",
        "price_overview",
        "genres_x",
        "tags",
        "features"
    ]
    return df.drop(columns=columns_to_drop, errors='ignore')

In [ ]:
# Drop unwanted columns
initial_processing = drop_unwanted_columns(initial_processing)

In [ ]:
# print out number of rows and columns
print('Rows:', initial_processing.shape[0])
print('Columns:', initial_processing.shape[1])

initial_processing.head()

Rows: 84453
Columns: 30


,steamid,id,name_x,price,reviews_x,followers,reviewScore,copiesSold,revenue,wishlists,avgPlaytime,developers_x,publishers_x,releaseDate,publisherClass,players,owners,required_age,is_free,detailed_description,about_the_game,short_description,supported_languages,pc_requirements,mac_requirements,linux_requirements,platforms,categories,genres_y,achievements
0,10,10,Counter-Strike,9.99,247199,215044,97,7021860,29305366.0,277200.0,183.686513,['Valve'],['Valve'],973054800000,AAA,11010395,16629587,0,False,Play the world's number 1 online action game. ...,Play the world's number 1 online action game. ...,Play the world's number 1 online action game. ...,"English<strong>*</strong>, French<strong>*</st...",{'minimum': '\n\t\t\t<p><strong>Minimum:</stro...,{'minimum': 'Minimum: OS X Snow Leopard 10.6....,"{'minimum': 'Minimum: Linux Ubuntu 12.04, Dual...","{'windows': True, 'mac': True, 'linux': True}","[{'id': 1, 'description': 'Multi-player'}, {'i...","[{'id': '1', 'description': 'Action'}]",NaN
1,20,20,Team Fortress Classic,4.99,8607,9617,87,375765,853393.0,37400.0,4.589017,['Valve'],['Valve'],922942800000,AAA,1415339,7823635,0,False,One of the most popular online action games of...,One of the most popular online action games of...,One of the most popular online action games of...,"English, French, German, Italian, Spanish - Sp...",{'minimum': '\n\t\t\t<p><strong>Minimum:</stro...,{'minimum': 'Minimum: OS X Snow Leopard 10.6....,"{'minimum': 'Minimum: Linux Ubuntu 12.04, Dual...","{'windows': True, 'mac': True, 'linux': True}","[{'id': 1, 'description': 'Multi-player'}, {'i...","[{'id': '1', 'description': 'Action'}]",NaN
2,30,30,Day of Defeat,4.99,7021,7244,90,230695,495765.0,37400.0,18.550989,['Valve'],['Valve'],1051761600000,AAA,1492839,6938005,0,False,Enlist in an intense brand of Axis vs. Allied ...,Enlist in an intense brand of Axis vs. Allied ...,Enlist in an intense brand of Axis vs. Allied ...,"English, French, German, Italian, Spanish - Spain",{'minimum': '\n\t\t\t<p><strong>Minimum:</stro...,{'minimum': 'Minimum: OS X Snow Leopard 10.6....,"{'minimum': 'Minimum: Linux Ubuntu 12.04, Dual...","{'windows': True, 'mac': True, 'linux': True}","[{'id': 1, 'description': 'Multi-player'}, {'i...","[{'id': '1', 'description': 'Action'}]",NaN
3,40,40,Deathmatch Classic,4.99,3136,2723,83,143370,310525.0,14300.0,2.748242,['Valve'],['Valve'],991368000000,AAA,1010152,7236021,0,False,Enjoy fast-paced multiplayer gaming with Death...,Enjoy fast-paced multiplayer gaming with Death...,Enjoy fast-paced multiplayer gaming with Death...,"English, French, German, Italian, Spanish - Sp...",{'minimum': '\n\t\t\t<p><strong>Minimum:</stro...,{'minimum': 'Minimum: OS X Snow Leopard 10.6....,"{'minimum': 'Minimum: Linux Ubuntu 12.04, Dual...","{'windows': True, 'mac': True, 'linux': True}","[{'id': 1, 'description': 'Multi-player'}, {'i...","[{'id': '1', 'description': 'Action'}]",NaN
4,50,50,Half-Life: Opposing Force,4.99,24809,10844,95,769170,1571654.0,51700.0,6.825604,['Gearbox Software'],['Valve'],941432400000,AAA,2076418,9806968,0,False,Return to the Black Mesa Research Facility as ...,Return to the Black Mesa Research Facility as ...,Return to the Black Mesa Research Facility as ...,"English, French, German, Korean",{'minimum': '\n\t\t\t<p><strong>Minimum:</stro...,{'minimum': 'Minimum: OS X Snow Leopard 10.6....,"{'minimum': 'Minimum: Linux Ubuntu 12.04, Dual...","{'windows': True, 'mac': True, 'linux': True}","[{'id': 2, 'description': 'Single-player'}, {'...","[{'id': '1', 'description': 'Action'}]",NaN


In [ ]:
# Inspect the data types of the columns
initial_processing.dtypes

,0
steamid,int64
id,int64
name_x,object
price,float64
reviews_x,int64
followers,int64
reviewScore,int64
copiesSold,int64
revenue,float64
wishlists,float64


In [ ]:
def convert_is_free_to_binary(df):
    df["is_free"] = df["is_free"].astype(int)  # Convert boolean to binary (0,1)
    return df

initial_processing = convert_is_free_to_binary(initial_processing).sort_values(by='steamid')

In [ ]:
# print out number of rows and columns
print('Rows:', initial_processing.shape[0])
print('Columns:', initial_processing.shape[1])

initial_processing.head()

Rows: 84453
Columns: 30


,steamid,id,name_x,price,reviews_x,followers,reviewScore,copiesSold,revenue,wishlists,avgPlaytime,developers_x,publishers_x,releaseDate,publisherClass,players,owners,required_age,is_free,detailed_description,about_the_game,short_description,supported_languages,pc_requirements,mac_requirements,linux_requirements,platforms,categories,genres_y,achievements
0,10,10,Counter-Strike,9.99,247199,215044,97,7021860,29305366.0,277200.0,183.686513,['Valve'],['Valve'],973054800000,AAA,11010395,16629587,0,0,Play the world's number 1 online action game. ...,Play the world's number 1 online action game. ...,Play the world's number 1 online action game. ...,"English<strong>*</strong>, French<strong>*</st...",{'minimum': '\n\t\t\t<p><strong>Minimum:</stro...,{'minimum': 'Minimum: OS X Snow Leopard 10.6....,"{'minimum': 'Minimum: Linux Ubuntu 12.04, Dual...","{'windows': True, 'mac': True, 'linux': True}","[{'id': 1, 'description': 'Multi-player'}, {'i...","[{'id': '1', 'description': 'Action'}]",NaN
1,20,20,Team Fortress Classic,4.99,8607,9617,87,375765,853393.0,37400.0,4.589017,['Valve'],['Valve'],922942800000,AAA,1415339,7823635,0,0,One of the most popular online action games of...,One of the most popular online action games of...,One of the most popular online action games of...,"English, French, German, Italian, Spanish - Sp...",{'minimum': '\n\t\t\t<p><strong>Minimum:</stro...,{'minimum': 'Minimum: OS X Snow Leopard 10.6....,"{'minimum': 'Minimum: Linux Ubuntu 12.04, Dual...","{'windows': True, 'mac': True, 'linux': True}","[{'id': 1, 'description': 'Multi-player'}, {'i...","[{'id': '1', 'description': 'Action'}]",NaN
2,30,30,Day of Defeat,4.99,7021,7244,90,230695,495765.0,37400.0,18.550989,['Valve'],['Valve'],1051761600000,AAA,1492839,6938005,0,0,Enlist in an intense brand of Axis vs. Allied ...,Enlist in an intense brand of Axis vs. Allied ...,Enlist in an intense brand of Axis vs. Allied ...,"English, French, German, Italian, Spanish - Spain",{'minimum': '\n\t\t\t<p><strong>Minimum:</stro...,{'minimum': 'Minimum: OS X Snow Leopard 10.6....,"{'minimum': 'Minimum: Linux Ubuntu 12.04, Dual...","{'windows': True, 'mac': True, 'linux': True}","[{'id': 1, 'description': 'Multi-player'}, {'i...","[{'id': '1', 'description': 'Action'}]",NaN
3,40,40,Deathmatch Classic,4.99,3136,2723,83,143370,310525.0,14300.0,2.748242,['Valve'],['Valve'],991368000000,AAA,1010152,7236021,0,0,Enjoy fast-paced multiplayer gaming with Death...,Enjoy fast-paced multiplayer gaming with Death...,Enjoy fast-paced multiplayer gaming with Death...,"English, French, German, Italian, Spanish - Sp...",{'minimum': '\n\t\t\t<p><strong>Minimum:</stro...,{'minimum': 'Minimum: OS X Snow Leopard 10.6....,"{'minimum': 'Minimum: Linux Ubuntu 12.04, Dual...","{'windows': True, 'mac': True, 'linux': True}","[{'id': 1, 'description': 'Multi-player'}, {'i...","[{'id': '1', 'description': 'Action'}]",NaN
4,50,50,Half-Life: Opposing Force,4.99,24809,10844,95,769170,1571654.0,51700.0,6.825604,['Gearbox Software'],['Valve'],941432400000,AAA,2076418,9806968,0,0,Return to the Black Mesa Research Facility as ...,Return to the Black Mesa Research Facility as ...,Return to the Black Mesa Research Facility as ...,"English, French, German, Korean",{'minimum': '\n\t\t\t<p><strong>Minimum:</stro...,{'minimum': 'Minimum: OS X Snow Leopard 10.6....,"{'minimum': 'Minimum: Linux Ubuntu 12.04, Dual...","{'windows': True, 'mac': True, 'linux': True}","[{'id': 2, 'description': 'Single-player'}, {'...","[{'id': '1', 'description': 'Action'}]",NaN


# **Clean the Supported Languages Column**

In [ ]:
initial_processing['supported_languages'].isnull().sum()

119

In [ ]:
print(initial_processing['supported_languages'][0])
initial_processing['supported_languages'].value_counts().head(10)

English<strong>*</strong>, French<strong>*</strong>, German<strong>*</strong>, Italian<strong>*</strong>, Spanish - Spain<strong>*</strong>, Simplified Chinese<strong>*</strong>, Traditional Chinese<strong>*</strong>, Korean<strong>*</strong><br><strong>*</strong>languages with full audio support


,count
supported_languages,
English,22743
English<strong>*</strong><br><strong>*</strong>languages with full audio support,18566
"English, Russian",1474
"English, Japanese",1184
"English, Simplified Chinese",1164
Simplified Chinese,998
Simplified Chinese<strong>*</strong><br><strong>*</strong>languages with full audio support,838
Japanese,583
"English<strong>*</strong>, Russian<strong>*</strong><br><strong>*</strong>languages with full audio support",485


In [ ]:
def process_language(df):
    """Process supported_languages column into a boolean 'is english' column."""
    df = df.copy()

    df["english"] = df["supported_languages"].apply(
        lambda x: 1 if "English" in str(x) else 0
    )
    df = df.drop("supported_languages", axis=1)

    return df


steam_dataset = process_language(initial_processing)
steam_dataset[["name_x", "english"]].head()

,name_x,english
0,Counter-Strike,1
1,Team Fortress Classic,1
2,Day of Defeat,1
3,Deathmatch Classic,1
4,Half-Life: Opposing Force,1


In [ ]:
steam_dataset['english'].value_counts()

,count
english,
1,80606
0,3847


In [ ]:
steam_dataset.head()

,steamid,id,name_x,price,reviews_x,followers,reviewScore,copiesSold,revenue,wishlists,avgPlaytime,developers_x,publishers_x,releaseDate,publisherClass,players,owners,required_age,is_free,detailed_description,about_the_game,short_description,pc_requirements,mac_requirements,linux_requirements,platforms,categories,genres_y,achievements,english
0,10,10,Counter-Strike,9.99,247199,215044,97,7021860,29305366.0,277200.0,183.686513,['Valve'],['Valve'],973054800000,AAA,11010395,16629587,0,0,Play the world's number 1 online action game. ...,Play the world's number 1 online action game. ...,Play the world's number 1 online action game. ...,{'minimum': '\n\t\t\t<p><strong>Minimum:</stro...,{'minimum': 'Minimum: OS X Snow Leopard 10.6....,"{'minimum': 'Minimum: Linux Ubuntu 12.04, Dual...","{'windows': True, 'mac': True, 'linux': True}","[{'id': 1, 'description': 'Multi-player'}, {'i...","[{'id': '1', 'description': 'Action'}]",NaN,1
1,20,20,Team Fortress Classic,4.99,8607,9617,87,375765,853393.0,37400.0,4.589017,['Valve'],['Valve'],922942800000,AAA,1415339,7823635,0,0,One of the most popular online action games of...,One of the most popular online action games of...,One of the most popular online action games of...,{'minimum': '\n\t\t\t<p><strong>Minimum:</stro...,{'minimum': 'Minimum: OS X Snow Leopard 10.6....,"{'minimum': 'Minimum: Linux Ubuntu 12.04, Dual...","{'windows': True, 'mac': True, 'linux': True}","[{'id': 1, 'description': 'Multi-player'}, {'i...","[{'id': '1', 'description': 'Action'}]",NaN,1
2,30,30,Day of Defeat,4.99,7021,7244,90,230695,495765.0,37400.0,18.550989,['Valve'],['Valve'],1051761600000,AAA,1492839,6938005,0,0,Enlist in an intense brand of Axis vs. Allied ...,Enlist in an intense brand of Axis vs. Allied ...,Enlist in an intense brand of Axis vs. Allied ...,{'minimum': '\n\t\t\t<p><strong>Minimum:</stro...,{'minimum': 'Minimum: OS X Snow Leopard 10.6....,"{'minimum': 'Minimum: Linux Ubuntu 12.04, Dual...","{'windows': True, 'mac': True, 'linux': True}","[{'id': 1, 'description': 'Multi-player'}, {'i...","[{'id': '1', 'description': 'Action'}]",NaN,1
3,40,40,Deathmatch Classic,4.99,3136,2723,83,143370,310525.0,14300.0,2.748242,['Valve'],['Valve'],991368000000,AAA,1010152,7236021,0,0,Enjoy fast-paced multiplayer gaming with Death...,Enjoy fast-paced multiplayer gaming with Death...,Enjoy fast-paced multiplayer gaming with Death...,{'minimum': '\n\t\t\t<p><strong>Minimum:</stro...,{'minimum': 'Minimum: OS X Snow Leopard 10.6....,"{'minimum': 'Minimum: Linux Ubuntu 12.04, Dual...","{'windows': True, 'mac': True, 'linux': True}","[{'id': 1, 'description': 'Multi-player'}, {'i...","[{'id': '1', 'description': 'Action'}]",NaN,1
4,50,50,Half-Life: Opposing Force,4.99,24809,10844,95,769170,1571654.0,51700.0,6.825604,['Gearbox Software'],['Valve'],941432400000,AAA,2076418,9806968,0,0,Return to the Black Mesa Research Facility as ...,Return to the Black Mesa Research Facility as ...,Return to the Black Mesa Research Facility as ...,{'minimum': '\n\t\t\t<p><strong>Minimum:</stro...,{'minimum': 'Minimum: OS X Snow Leopard 10.6....,"{'minimum': 'Minimum: Linux Ubuntu 12.04, Dual...","{'windows': True, 'mac': True, 'linux': True}","[{'id': 2, 'description': 'Single-player'}, {'...","[{'id': '1', 'description': 'Action'}]",NaN,1


# **Age**

In [ ]:
steam_df = steam_dataset.copy()

steam_df['required_age'].value_counts()

,count
required_age,
0,43440
0,39952
17,434
17,148
18,119
13,98
18,76
16,47
13,38


In [ ]:
def clean_required_age(df):
    """Removes '17+', '18+', and any non-numeric values from the required_age column."""
    df['required_age'] = pd.to_numeric(df['required_age'], errors='coerce')  # Convert to numeric
    df = df.dropna(subset=['required_age'])  # Drop non-numeric values
    df['required_age'] = df['required_age'].astype(int)  # Ensure integer type
    return df

# Apply the cleaning function
steam_df = clean_required_age(steam_df)

# Now you can safely process the age data
print(steam_df['required_age'].value_counts().sort_index())

required_age
0     83392
1         1
3         5
6         3
7         7
10       24
11        1
12       28
13      136
14        1
15        8
16       66
17      582
18      195
20        1
Name: count, dtype: int64


<ipython-input-24-91bf4636284f>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['required_age'] = df['required_age'].astype(int)  # Ensure integer type


In [ ]:
steam_df['required_age'].value_counts()

,count
required_age,
0,83392
17,582
18,195
13,136
16,66
12,28
10,24
15,8
7,7


In [ ]:
def process_age(df):
    """Format ratings in age column to be in line with the PEGI Age Ratings system."""
    # PEGI Age ratings: 3, 7, 12, 16, 18
    cut_points = [-1, 0, 3, 7, 12, 16, 2000]
    label_values = [0, 3, 7, 12, 16, 18]

    df['required_age'] = pd.cut(df['required_age'], bins=cut_points, labels=label_values)

    return df


age_df = process_age(steam_df)
age_df['required_age'].value_counts().sort_index()

,count
required_age,
0,83392
3,6
7,10
12,53
16,211
18,778


## Processing the Platforms Column

In [ ]:
age_df['platforms'].head()

,platforms
0,"{'windows': True, 'mac': True, 'linux': True}"
1,"{'windows': True, 'mac': True, 'linux': True}"
2,"{'windows': True, 'mac': True, 'linux': True}"
3,"{'windows': True, 'mac': True, 'linux': True}"
4,"{'windows': True, 'mac': True, 'linux': True}"


In [ ]:
platforms_first_row = age_df['platforms'].iloc[0]

print(type(platforms_first_row))

platforms_first_row

<class 'str'>


"{'windows': True, 'mac': True, 'linux': True}"

In [ ]:
eval_first_row = literal_eval(platforms_first_row)

print(type(eval_first_row))
print(eval_first_row)

eval_first_row['windows']

<class 'dict'>
{'windows': True, 'mac': True, 'linux': True}


True

In [ ]:
age_df['platforms'].isnull().sum()

0

In [ ]:
# create string of keys, joined on a semi-colon
';'.join(eval_first_row.keys())

'windows;mac;linux'

In [ ]:
platforms = {'windows': True, 'mac': True, 'linux': False}

# list comprehension
print([x for x in platforms.keys() if platforms[x]])

# using list comprehension in join
';'.join(x for x in platforms.keys() if platforms[x])

['windows', 'mac']


'windows;mac'

In [ ]:
def process_platforms(df):
    """Split platforms column into separate boolean columns for each platform."""
    # evaluate values in platforms column, so can index into dictionaries
    df = df.copy()

    def parse_platforms(x):

        d = literal_eval(x)

        return ';'.join(platform for platform in d.keys() if d[platform])

    df['platforms'] = df['platforms'].apply(parse_platforms)

    return df


platforms_df = process_platforms(age_df)
platforms_df['platforms'].value_counts()


,count
platforms,
windows,64527
windows;mac;linux,9048
windows;mac,7802
windows;linux,3056
mac,11
linux,5
mac;linux,1


In [ ]:
def process_platforms(df):
    """Split platforms column into separate boolean columns for each platform."""
    df = df.copy()

    def parse_platforms(x):
        d = ast.literal_eval(x)
        return {platform: int(d.get(platform, False)) for platform in ['windows', 'mac', 'linux']}

    # Apply function and create new columns
    platform_data = df['platforms'].apply(parse_platforms).apply(pd.Series)

    # Merge with original dataframe (removing the old 'platforms' column)
    df = df.drop(columns=['platforms']).join(platform_data)

    return df

# Apply function and check the 3 columns
platforms_df = process_platforms(age_df)

In [ ]:
platforms_df['windows'].value_counts()

,count
windows,
1,84433
0,17


platforms_df[',ac'].value_counts()

In [ ]:
platforms_df['mac'].value_counts()

,count
mac,
0,67588
1,16862


In [ ]:
platforms_df['linux'].value_counts()

,count
linux,
0,72340
1,12110


In [ ]:
# print out number of rows and columns
print('Rows:', platforms_df.shape[0])
print('Columns:', platforms_df.shape[1])

platforms_df.head()

Rows: 84450
Columns: 32


,steamid,id,name_x,price,reviews_x,followers,reviewScore,copiesSold,revenue,wishlists,avgPlaytime,developers_x,publishers_x,releaseDate,publisherClass,players,owners,required_age,is_free,detailed_description,about_the_game,short_description,pc_requirements,mac_requirements,linux_requirements,categories,genres_y,achievements,english,windows,mac,linux
0,10,10,Counter-Strike,9.99,247199,215044,97,7021860,29305366.0,277200.0,183.686513,['Valve'],['Valve'],973054800000,AAA,11010395,16629587,0,0,Play the world's number 1 online action game. ...,Play the world's number 1 online action game. ...,Play the world's number 1 online action game. ...,{'minimum': '\n\t\t\t<p><strong>Minimum:</stro...,{'minimum': 'Minimum: OS X Snow Leopard 10.6....,"{'minimum': 'Minimum: Linux Ubuntu 12.04, Dual...","[{'id': 1, 'description': 'Multi-player'}, {'i...","[{'id': '1', 'description': 'Action'}]",NaN,1,1,1,1
1,20,20,Team Fortress Classic,4.99,8607,9617,87,375765,853393.0,37400.0,4.589017,['Valve'],['Valve'],922942800000,AAA,1415339,7823635,0,0,One of the most popular online action games of...,One of the most popular online action games of...,One of the most popular online action games of...,{'minimum': '\n\t\t\t<p><strong>Minimum:</stro...,{'minimum': 'Minimum: OS X Snow Leopard 10.6....,"{'minimum': 'Minimum: Linux Ubuntu 12.04, Dual...","[{'id': 1, 'description': 'Multi-player'}, {'i...","[{'id': '1', 'description': 'Action'}]",NaN,1,1,1,1
2,30,30,Day of Defeat,4.99,7021,7244,90,230695,495765.0,37400.0,18.550989,['Valve'],['Valve'],1051761600000,AAA,1492839,6938005,0,0,Enlist in an intense brand of Axis vs. Allied ...,Enlist in an intense brand of Axis vs. Allied ...,Enlist in an intense brand of Axis vs. Allied ...,{'minimum': '\n\t\t\t<p><strong>Minimum:</stro...,{'minimum': 'Minimum: OS X Snow Leopard 10.6....,"{'minimum': 'Minimum: Linux Ubuntu 12.04, Dual...","[{'id': 1, 'description': 'Multi-player'}, {'i...","[{'id': '1', 'description': 'Action'}]",NaN,1,1,1,1
3,40,40,Deathmatch Classic,4.99,3136,2723,83,143370,310525.0,14300.0,2.748242,['Valve'],['Valve'],991368000000,AAA,1010152,7236021,0,0,Enjoy fast-paced multiplayer gaming with Death...,Enjoy fast-paced multiplayer gaming with Death...,Enjoy fast-paced multiplayer gaming with Death...,{'minimum': '\n\t\t\t<p><strong>Minimum:</stro...,{'minimum': 'Minimum: OS X Snow Leopard 10.6....,"{'minimum': 'Minimum: Linux Ubuntu 12.04, Dual...","[{'id': 1, 'description': 'Multi-player'}, {'i...","[{'id': '1', 'description': 'Action'}]",NaN,1,1,1,1
4,50,50,Half-Life: Opposing Force,4.99,24809,10844,95,769170,1571654.0,51700.0,6.825604,['Gearbox Software'],['Valve'],941432400000,AAA,2076418,9806968,0,0,Return to the Black Mesa Research Facility as ...,Return to the Black Mesa Research Facility as ...,Return to the Black Mesa Research Facility as ...,{'minimum': '\n\t\t\t<p><strong>Minimum:</stro...,{'minimum': 'Minimum: OS X Snow Leopard 10.6....,"{'minimum': 'Minimum: Linux Ubuntu 12.04, Dual...","[{'id': 2, 'description': 'Single-player'}, {'...","[{'id': '1', 'description': 'Action'}]",NaN,1,1,1,1


## Processing Developers and Publishers

In [ ]:
print('Developers null counts:', platforms_df['developers_x'].isnull().sum())
print('Developers empty list counts:', platforms_df[platforms_df['developers_x'] == "['']"].shape[0])

print('\nPublishers null counts:', platforms_df['publishers_x'].isnull().sum())
print('Publishers empty list counts:', platforms_df[platforms_df['publishers_x'] == "['']"].shape[0])

Developers null counts: 0
Developers empty list counts: 1

Publishers null counts: 0
Publishers empty list counts: 5


In [ ]:
def print_steam_links(df):
    """Print links to store page for apps in a dataframe."""
    url_base = "https://store.steampowered.com/app/"

    for i, row in df.iterrows():
        appid = row['steamid']
        name = row['name_x']

        print(name + ':', url_base + str(appid))

In [ ]:
no_dev = platforms_df[platforms_df['developers_x'].isnull()]

print('Total games missing developer:', no_dev.shape[0], '\n')

print_steam_links(no_dev[:5])

no_pub = platforms_df[platforms_df['publishers_x'] == "['']"]

print('\nTotal games missing publisher:', no_pub.shape[0], '\n')
print_steam_links(no_pub[:5])

no_dev_or_pub = platforms_df[(platforms_df['developers_x'].isnull()) & (platforms_df['publishers_x'] == "['']")]

print('\nTotal games missing developer and publisher:', no_dev_or_pub.shape[0], '\n')
print_steam_links(no_dev_or_pub[:5])

Total games missing developer: 0 


Total games missing publisher: 5 

Time Rifters: https://store.steampowered.com/app/270010
Time Clickers: https://store.steampowered.com/app/385770
Gamma Blast: https://store.steampowered.com/app/759830
Sharp: https://store.steampowered.com/app/889530
Achtung die Kugel!: https://store.steampowered.com/app/938660

Total games missing developer and publisher: 0 



In [ ]:
platforms_df[['developers_x', 'publishers_x']].iloc[24:28]

,developers_x,publishers_x
24,['Ritual Entertainment'],['Ritual Entertainment']
25,"['Ritual Entertainment', 'Nightdive Studios']",['Nightdive Studios']
26,['Introversion Software'],['Introversion Software']
27,['Introversion Software'],['Introversion Software']


In [ ]:
def process_developers_and_publishers(df):
    # remove rows with missing data
    df = df[(df['developers_x'].notnull()) & (df['publishers_x'] != "['']")].copy()

    for col in ['developers_x', 'publishers_x']:
        df[col] = df[col].apply(lambda x: literal_eval(x))

        # filter dataframe to rows with lists longer than 1, and store the number of rows
        num_rows = df[df[col].str.len() > 1].shape[0]

        print('Rows in {} column with multiple values:'.format(col), num_rows)

process_developers_and_publishers(platforms_df)

Rows in developers_x column with multiple values: 5869
Rows in publishers_x column with multiple values: 3102


In [ ]:
', '.join(['one item'])

'one item'

In [ ]:
', '.join(['multiple', 'different', 'items'])

'multiple, different, items'

In [ ]:
platforms_df.loc[platforms_df['developers_x'].str.contains(",", na=False), ['steamid', 'developers_x', 'publishers_x']].head(4)

,steamid,developers_x,publishers_x
25,1313,"['Ritual Entertainment', 'Nightdive Studios']",['Nightdive Studios']
42,2210,"['RavenSoft', 'id Software']",['Bethesda Softworks']
44,2280,"['id Software', 'Nightdive Studios']",['Bethesda']
45,2310,"['id Software', 'Nightdive Studios', 'MachineG...",['Bethesda Softworks']


In [ ]:
platforms_df.loc[platforms_df['developers_x'].str.contains(";", na=False), ['steamid', 'developers_x', 'publishers_x']]

,steamid,developers_x,publishers_x
12060,568480,"[';)', 'Quickdraw Studios']",['Quickdraw Studios']
60910,1999920,['7th;MINT'],['DoubleChallenge Inc.']
63896,2119440,['MasterGames Co;Ltd;'],['MasterGames Co;Ltd;']
79445,2686590,"['Husky-Leaf Studios', 'Sku;Te']",['Husky-Leaf Studios']
87413,3036330,['MasterGames Co;Ltd;'],['MasterGames Co;Ltd;']


In [ ]:
platforms_df[(platforms_df['publishers_x'] == "['NA']") | (platforms_df['publishers_x'] == "['N/A']")].shape[0]

29

In [ ]:
def process_developers_and_publishers(df):
    """Parse columns as semicolon-separated string."""
    # remove rows with missing data (~ means not)
    df = df[(df['developers_x'].notnull()) & (df['publishers_x'] != "['']")].copy()
    df = df[~(df['developers_x'].str.contains(';')) & ~(df['publishers_x'].str.contains(';'))]
    df = df[(df['publishers_x'] != "['NA']") & (df['publishers_x'] != "['N/A']")]

    # create list for each
    df['developer'] = df['developers_x'].apply(lambda x: ';'.join(literal_eval(x)))
    df['publisher'] = df['publishers_x'].apply(lambda x: ';'.join(literal_eval(x)))

    df = df.drop(['developers_x', 'publishers_x'], axis=1)

    return df

dev_pub_df = process_developers_and_publishers(platforms_df)
dev_pub_df[['name_x', 'steamid', 'developer', 'publisher']].head()

,name_x,steamid,developer,publisher
0,Counter-Strike,10,Valve,Valve
1,Team Fortress Classic,20,Valve,Valve
2,Day of Defeat,30,Valve,Valve
3,Deathmatch Classic,40,Valve,Valve
4,Half-Life: Opposing Force,50,Gearbox Software,Valve


In [ ]:
print('Developers null counts:', dev_pub_df['developer'].isnull().sum())
print('Developers empty list counts:', dev_pub_df[dev_pub_df['developer'] == "['']"].shape[0])

print('\nPublishers null counts:', dev_pub_df['publisher'].isnull().sum())
print('Publishers empty list counts:', dev_pub_df[dev_pub_df['publisher'] == "['']"].shape[0])

Developers null counts: 0
Developers empty list counts: 0

Publishers null counts: 0
Publishers empty list counts: 0


## Processing Categories and Genres

In [ ]:
print('Categories:\n')
print('Null values:', dev_pub_df['categories'].isnull().sum())
print()
print(dev_pub_df['categories'][0])

print('\nGenres:\n')
print('Null values:', dev_pub_df['genres_y'].isnull().sum())
print()
print(dev_pub_df['genres_y'].iloc[0])
print(dev_pub_df['genres_y'].iloc[1000])

Categories:

Null values: 20

[{'id': 1, 'description': 'Multi-player'}, {'id': 49, 'description': 'PvP'}, {'id': 36, 'description': 'Online PvP'}, {'id': 37, 'description': 'Shared/Split Screen PvP'}, {'id': 8, 'description': 'Valve Anti-Cheat enabled'}, {'id': 62, 'description': 'Family Sharing'}]

Genres:

Null values: 138

[{'id': '1', 'description': 'Action'}]
[{'id': '3', 'description': 'RPG'}]


In [ ]:
print_steam_links(dev_pub_df[dev_pub_df['categories'].isnull()].sample(5, random_state=0))

Cat Purrtrol: Find All 100!: https://store.steampowered.com/app/2895990
Down To One Dedicated Server: https://store.steampowered.com/app/345950
画笔勇者 | Paintbrush Warrior: https://store.steampowered.com/app/3237590
DayZ Tools: https://store.steampowered.com/app/830640
SteamWorld Quest: Hand of Gilgamech - Soundtrack: https://store.steampowered.com/app/1085590


In [ ]:
print_steam_links(dev_pub_df[dev_pub_df['genres_y'].isnull()].sample(5, random_state=0))

Sonic the Hedgehog 4 - Episode I: https://store.steampowered.com/app/202530
Cute Honey: https://store.steampowered.com/app/1347430
Antagonist: https://store.steampowered.com/app/585260
Burger Kombat: https://store.steampowered.com/app/2487680
Cobalt Dedicated Server: https://store.steampowered.com/app/549990


In [ ]:
example_category = "[{'id': 1, 'description': 'Multi-player'}, {'id': 36, 'description': 'Online Multi-Player'}, {'id': 37, 'description': 'Local Multi-Player'}]"

[x['description'] for x in literal_eval(example_category)]

['Multi-player', 'Online Multi-Player', 'Local Multi-Player']

In [ ]:
def process_categories_and_genres(df):
    df = df.copy()
    df = df[(df['categories'].notnull()) & (df['genres_y'].notnull())]

    for col in ['categories', 'genres_y']:
        df[col] = df[col].apply(lambda x: ';'.join(item['description'] for item in literal_eval(x)))

    return df

cat_gen_df = process_categories_and_genres(dev_pub_df)
cat_gen_df[['steamid', 'categories', 'genres_y']].head()

,steamid,categories,genres_y
0,10,Multi-player;PvP;Online PvP;Shared/Split Scree...,Action
1,20,Multi-player;PvP;Online PvP;Shared/Split Scree...,Action
2,30,Multi-player;Valve Anti-Cheat enabled;Family S...,Action
3,40,Multi-player;PvP;Online PvP;Shared/Split Scree...,Action
4,50,Single-player;Multi-player;Valve Anti-Cheat en...,Action


## Processing Achievements and Content Descriptors

In [ ]:
print('Achievements null counts:', cat_gen_df['achievements'].isnull().sum())
# print('Content Decsriptors null counts:', cat_gen_df['content_descriptors'].isnull().sum())

cat_gen_df[['name_x', 'achievements']].iloc[8:13]

Achievements null counts: 36056


,name_x,achievements
8,Half-Life: Blue Shift,NaN
9,Half-Life 2,"{'total': 69, 'highlighted': [{'name': 'Defian..."
10,Counter-Strike: Source,"{'total': 147, 'highlighted': [{'name': 'Someo..."
11,Day of Defeat: Source,"{'total': 54, 'highlighted': [{'name': 'Double..."
12,Half-Life Deathmatch: Source,NaN


In [ ]:
literal_eval(cat_gen_df['achievements'][9])

{'total': 69,
 'highlighted': [{'name': 'Defiant',
   'path': 'https://cdn.akamai.steamstatic.com/steamcommunity/public/images/apps/220/hl2_hit_cancop_withcan.jpg'},
  {'name': 'Submissive',
   'path': 'https://cdn.akamai.steamstatic.com/steamcommunity/public/images/apps/220/hl2_put_canintrash.jpg'},
  {'name': 'Malcontent',
   'path': 'https://cdn.akamai.steamstatic.com/steamcommunity/public/images/apps/220/hl2_escape_apartmentraid.jpg'},
  {'name': 'What cat?',
   'path': 'https://cdn.akamai.steamstatic.com/steamcommunity/public/images/apps/220/hl2_break_miniteleporter.jpg'},
  {'name': 'Trusty Hardware',
   'path': 'https://cdn.akamai.steamstatic.com/steamcommunity/public/images/apps/220/hl2_get_crowbar.jpg'},
  {'name': 'Barnacle Bowling',
   'path': 'https://cdn.akamai.steamstatic.com/steamcommunity/public/images/apps/220/hl2_kill_barnacleswithbarrel.jpg'},
  {'name': "Anchor's Aweigh!",
   'path': 'https://cdn.akamai.steamstatic.com/steamcommunity/public/images/apps/220/hl2_get_a

In [ ]:
def process_achievements_and_descriptors(df):
    """Parse as total number of achievements."""
    df = df.copy()

    def parse_achievements(x):
        if x is np.nan:
            # missing data, assume has no achievements
            return 0
        else:
            # else has data, so can extract and return number under total
            return literal_eval(x)['total']

    df['achievements'] = df['achievements'].apply(parse_achievements)

    return df

achiev_df = process_achievements_and_descriptors(cat_gen_df)
achiev_df['achievements'].value_counts().head()

,count
achievements,
0,36762
10,2390
12,1832
20,1735
6,1617


In [ ]:
# print out number of rows and columns
print('Rows:', achiev_df.shape[0])
print('Columns:', achiev_df.shape[1])

achiev_df.head()

Rows: 84235
Columns: 32


,steamid,id,name_x,price,reviews_x,followers,reviewScore,copiesSold,revenue,wishlists,avgPlaytime,releaseDate,publisherClass,players,owners,required_age,is_free,detailed_description,about_the_game,short_description,pc_requirements,mac_requirements,linux_requirements,categories,genres_y,achievements,english,windows,mac,linux,developer,publisher
0,10,10,Counter-Strike,9.99,247199,215044,97,7021860,29305366.0,277200.0,183.686513,973054800000,AAA,11010395,16629587,0,0,Play the world's number 1 online action game. ...,Play the world's number 1 online action game. ...,Play the world's number 1 online action game. ...,{'minimum': '\n\t\t\t<p><strong>Minimum:</stro...,{'minimum': 'Minimum: OS X Snow Leopard 10.6....,"{'minimum': 'Minimum: Linux Ubuntu 12.04, Dual...",Multi-player;PvP;Online PvP;Shared/Split Scree...,Action,0,1,1,1,1,Valve,Valve
1,20,20,Team Fortress Classic,4.99,8607,9617,87,375765,853393.0,37400.0,4.589017,922942800000,AAA,1415339,7823635,0,0,One of the most popular online action games of...,One of the most popular online action games of...,One of the most popular online action games of...,{'minimum': '\n\t\t\t<p><strong>Minimum:</stro...,{'minimum': 'Minimum: OS X Snow Leopard 10.6....,"{'minimum': 'Minimum: Linux Ubuntu 12.04, Dual...",Multi-player;PvP;Online PvP;Shared/Split Scree...,Action,0,1,1,1,1,Valve,Valve
2,30,30,Day of Defeat,4.99,7021,7244,90,230695,495765.0,37400.0,18.550989,1051761600000,AAA,1492839,6938005,0,0,Enlist in an intense brand of Axis vs. Allied ...,Enlist in an intense brand of Axis vs. Allied ...,Enlist in an intense brand of Axis vs. Allied ...,{'minimum': '\n\t\t\t<p><strong>Minimum:</stro...,{'minimum': 'Minimum: OS X Snow Leopard 10.6....,"{'minimum': 'Minimum: Linux Ubuntu 12.04, Dual...",Multi-player;Valve Anti-Cheat enabled;Family S...,Action,0,1,1,1,1,Valve,Valve
3,40,40,Deathmatch Classic,4.99,3136,2723,83,143370,310525.0,14300.0,2.748242,991368000000,AAA,1010152,7236021,0,0,Enjoy fast-paced multiplayer gaming with Death...,Enjoy fast-paced multiplayer gaming with Death...,Enjoy fast-paced multiplayer gaming with Death...,{'minimum': '\n\t\t\t<p><strong>Minimum:</stro...,{'minimum': 'Minimum: OS X Snow Leopard 10.6....,"{'minimum': 'Minimum: Linux Ubuntu 12.04, Dual...",Multi-player;PvP;Online PvP;Shared/Split Scree...,Action,0,1,1,1,1,Valve,Valve
4,50,50,Half-Life: Opposing Force,4.99,24809,10844,95,769170,1571654.0,51700.0,6.825604,941432400000,AAA,2076418,9806968,0,0,Return to the Black Mesa Research Facility as ...,Return to the Black Mesa Research Facility as ...,Return to the Black Mesa Research Facility as ...,{'minimum': '\n\t\t\t<p><strong>Minimum:</stro...,{'minimum': 'Minimum: OS X Snow Leopard 10.6....,"{'minimum': 'Minimum: Linux Ubuntu 12.04, Dual...",Single-player;Multi-player;Valve Anti-Cheat en...,Action,0,1,1,1,1,Gearbox Software,Valve


## Export Columns

In [ ]:
achiev_df[['detailed_description', 'about_the_game', 'short_description']].isnull().sum()

,0
detailed_description,12
about_the_game,13
short_description,13


In [ ]:
achiev_df[achiev_df['detailed_description'].isnull()].head(3)

,steamid,id,name_x,price,reviews_x,followers,reviewScore,copiesSold,revenue,wishlists,avgPlaytime,releaseDate,publisherClass,players,owners,required_age,is_free,detailed_description,about_the_game,short_description,pc_requirements,mac_requirements,linux_requirements,categories,genres_y,achievements,english,windows,mac,linux,developer,publisher
35556,1231920,1231920,Convoy Mod Tools,0.00,1,0,100,15,0.0,NaN,0.000000,1579582800000,Indie,15,15,0,0,NaN,NaN,NaN,[],[],[],Single-player;Family Sharing,Action;Adventure;Indie;Strategy,0,1,1,0,0,Convoy Games,Indietopia Games
35799,1238470,1238470,Come on-be with you,0.99,5,38,20,18,22.0,1100.0,0.537046,1581570000000,Hobbyist,90,90,0,0,NaN,NaN,NaN,{'minimum': '<strong>Minimum:</strong><br><ul ...,[],[],Single-player;Family Sharing,Action;Casual;Indie;Simulation;Sports;Strategy,0,1,0,0,1,yu,yun
40051,1366270,1366270,Primal Carnage: Extinction - Workshop Tool,0.00,2,35229,100,68,0.0,422748.0,0.444059,1594958400000,Indie,68,68,0,0,NaN,NaN,NaN,[],[],[],Single-player,Action,0,0,1,0,0,Circle 5 Studios,Circle 5 Studios


In [ ]:
achiev_df[achiev_df['detailed_description'].str.len() <= 20]

,steamid,id,name_x,price,reviews_x,followers,reviewScore,copiesSold,revenue,wishlists,avgPlaytime,releaseDate,publisherClass,players,owners,required_age,is_free,detailed_description,about_the_game,short_description,pc_requirements,mac_requirements,linux_requirements,categories,genres_y,achievements,english,windows,mac,linux,developer,publisher
4510,342570,342570,HIS (Heroes In the Sky),0.00,554,2517,37,94162,0.0,6600.0,2.585608,1425358800000,Hobbyist,94162,303410,0,1,-,-,-,{'minimum': '<strong>Minimum:</strong><br><ul ...,[],[],Partial Controller Support,Indie,1,0,1,0,0,GameUS Inc.,GameUS Inc.
16507,698950,698950,Chaos and the White Robot,0.99,26,82,73,424,674.0,2200.0,6.217809,1505448000000,Hobbyist,1577,1577,0,0,[no description],[no description],[no description],{'minimum': '<strong>Minimum:</strong><br><ul ...,[],[],Single-player;Partial Controller Support;Famil...,Action;Casual;Indie,0,1,1,0,0,SDF games,SDF games
20128,797660,797660,God Test,0.00,14,191,43,1237,0.0,2292.0,1.163577,1524024000000,Hobbyist,1237,17930,0,1,God Test,God Test,God Test,{'minimum': '<strong>Minimum:</strong><br><ul ...,[],[],Multi-player;MMO;PvP;Online PvP;Co-op;Online C...,Action;Massively Multiplayer;RPG;Strategy,0,1,1,0,0,God Test,God Test
21198,828080,828080,Pong like,0.00,8,14,0,270,0.0,1100.0,0.163738,1523851200000,Hobbyist,270,270,0,0,a simple game,a simple game,"In Pong like, the player will use the mouse to...",{'minimum': '<strong>Minimum:</strong><br><ul ...,[],[],Single-player;Family Sharing,Action;Indie,0,1,1,0,0,树册,树册
35245,1222080,1222080,Feeling Square?,0.99,2,10,50,18,16.0,120.0,0.997223,1606366800000,Hobbyist,36,36,0,0,COMING SOON....,COMING SOON....,Something huge is Coming Soon...,{'minimum': '<strong>Minimum:</strong><br><ul ...,[],[],Single-player;Steam Achievements;Family Sharing,Action;Indie;Strategy,0,1,1,0,0,MikoTech;SquareBoxStudio,MikoTech
43433,1463180,1463180,九个世界（Nine worlds）,0.99,2,76,50,36,524.0,912.0,0.368333,1630728000000,Hobbyist,36,36,0,0,Project interruption,Project interruption,"This is a ARPG game, at the same time contains...",{'minimum': '<strong>Minimum:</strong><br><ul ...,{'minimum': '<strong>Minimum:</strong><br><ul ...,{'minimum': '<strong>Minimum:</strong><br><ul ...,Single-player;Partial Controller Support;Famil...,Action;Adventure;Indie;RPG;Strategy,0,1,1,0,0,登,登
59888,1967260,1967260,Double Dragon Gaiden: Rise Of The Dragons,24.99,990,8744,85,29872,464111.0,126500.0,7.241864,1690430400000,Indie,41476,41476,0,0,<h1>DISCORD</h1><p>,"<img class=""bb_img"" src=""https://shared.akamai...",The Double Dragon brothers return in this fres...,{'minimum': '<strong>Minimum:</strong><br><ul ...,{'minimum': '<strong>Minimum:</strong><br><ul ...,{'minimum': '<strong>Minimum:</strong><br><ul ...,Single-player;Multi-player;Co-op;Online Co-op;...,Action;Adventure;Casual,20,1,1,0,0,Secret Base,Maximum Entertainment;Joystick
77969,2627040,2627040,Uncontained,4.99,0,56,0,1,4.0,672.0,0.000000,1699592400000,Hobbyist,1,1,0,0,Nothing here..,Nothing here..,Nothing here..,{'minimum': '<strong>Minimum:</strong><br><ul ...,[],[],Multi-player;PvP;Online PvP;LAN PvP;Co-op;Onli...,Action;Indie;Strategy,0,1,1,0,0,Nuallual Entertainment,Nuallual Entertainment
86210,2978750,2978750,DeMonkey,0.99,6,19,33,79,48.0,228.0,0.506234,1719115200000,Hobbyist,79,79,0,0,nothing,nothing,nothing,[],[],[],Multi-player;Steam Cloud;Family Sharing,Action;Adventure;Racing,0,0,1,0,0,DenelVR;twixs,DenelVR
88663,3096730,3096730,Postponed,0.00,97,363,78,66856,0.0,1100.0,1.482191,1723420800000,Hobbyist,66865,66865,0,1,Postponed,Postponed,Postponed,{'minimum': '<strong>Minimum:</strong><br><ul ...,[],[],Single-player,Action;Indie;Free To Play,0,1,1,0,0,Postponed,Postponed


In [ ]:
def export_data(df, filename):
    """Export dataframe to csv file, filename prepended with 'steam_'.

    filename : str without file extension
    """
    filepath = f'{dataset_final_dir}/steam_' + filename + '.csv'

    df.to_csv(filepath, index=False)

    print_name = filename.replace('_', ' ')
    print("Exported {} to '{}'".format(print_name, filepath))

In [ ]:
def process_descriptions(df, export=False):
    """Export descriptions to external csv file then remove these columns."""
    # remove rows with missing description data
    df = df[df['detailed_description'].notnull()].copy()

    # remove rows with unusually small description
    df = df[df['detailed_description'].str.len() > 20]

    # by default we don't export, useful if calling function later
    if export:
        # create dataframe of description columns
        description_data = df[['steamid', 'detailed_description', 'about_the_game', 'short_description']]

        export_data(description_data, filename='description_data')

    # drop description columns from main dataframe
    df = df.drop(['detailed_description', 'about_the_game', 'short_description'], axis=1)

    return df

desc_df = process_descriptions(achiev_df, export=True)

Exported description data to '/content/drive/MyDrive/Colab Notebooks/steamdata/experimental/datasets_final/steam_description_data.csv'


In [ ]:
# inspect exported data
pd.read_csv(f'{dataset_final_dir}/steam_description_data.csv').head()

,steamid,detailed_description,about_the_game,short_description
0,10,Play the world's number 1 online action game. ...,Play the world's number 1 online action game. ...,Play the world's number 1 online action game. ...
1,20,One of the most popular online action games of...,One of the most popular online action games of...,One of the most popular online action games of...
2,30,Enlist in an intense brand of Axis vs. Allied ...,Enlist in an intense brand of Axis vs. Allied ...,Enlist in an intense brand of Axis vs. Allied ...
3,40,Enjoy fast-paced multiplayer gaming with Death...,Enjoy fast-paced multiplayer gaming with Death...,Enjoy fast-paced multiplayer gaming with Death...
4,50,Return to the Black Mesa Research Facility as ...,Return to the Black Mesa Research Facility as ...,Return to the Black Mesa Research Facility as ...


In [ ]:
print('Before removing data:\n')
achiev_df.info(verbose=False, memory_usage="deep")

print('\nData with descriptions and media removed:\n')
desc_df.info(verbose=False, memory_usage="deep")

Before removing data:

<class 'pandas.core.frame.DataFrame'>
Index: 84235 entries, 0 to 93840
Columns: 32 entries, steamid to publisher
dtypes: category(1), float64(4), int64(15), object(12)
memory usage: 610.7 MB

Data with descriptions and media removed:

<class 'pandas.core.frame.DataFrame'>
Index: 84213 entries, 0 to 93840
Columns: 29 entries, steamid to publisher
dtypes: category(1), float64(4), int64(15), object(9)
memory usage: 153.3 MB


### System Requirements

In [ ]:
requirements_cols = ['pc_requirements', 'mac_requirements', 'linux_requirements']

print('null counts:\n')

for col in requirements_cols:
    print(col+':', desc_df[col].isnull().sum())

null counts:

pc_requirements: 0
mac_requirements: 0
linux_requirements: 0


In [ ]:
desc_df[['steamid', 'pc_requirements', 'mac_requirements', 'linux_requirements']].tail()

,steamid,pc_requirements,mac_requirements,linux_requirements
93836,3483220,{'minimum': '<strong>Minimum:</strong><br><ul ...,[],[]
93837,3484130,{'minimum': '<strong>Minimum:</strong><br><ul ...,[],[]
93838,3487130,{'minimum': '<strong>Minimum:</strong><br><ul ...,[],[]
93839,3495070,{'minimum': '<strong>Minimum:</strong><br><ul ...,[],[]
93840,3504260,{'minimum': '<strong>Minimum:</strong><br><ul ...,{'minimum': '<strong>Minimum:</strong><br><ul ...,[]


In [ ]:
print('Empty list counts:\n')

for col in requirements_cols:
    print(col+':', desc_df[desc_df[col] == '[]'].shape[0])

Empty list counts:

pc_requirements: 26
mac_requirements: 44308
linux_requirements: 49228


In [ ]:
for col in ['mac_requirements', 'linux_requirements']:
    platform = col.split('_')[0]  # Extract platform name (mac or linux)
    print(f"{platform}:", desc_df[(desc_df[col] == '[]') & (desc_df[platform] == 1)].shape[0])

mac: 428
linux: 447


In [ ]:
print('windows:', desc_df[(desc_df['pc_requirements'] == '[]') & (desc_df['windows'])].shape[0])

windows: 16


In [ ]:
missing_windows_requirements = desc_df[(desc_df['pc_requirements'] == '[]') & (desc_df['windows'])]

print_steam_links(missing_windows_requirements[:5])

Uplink: https://store.steampowered.com/app/1510
Battlestations: Midway: https://store.steampowered.com/app/6870
Grand Theft Auto 2: https://store.steampowered.com/app/12180
World of Goo: https://store.steampowered.com/app/22000
Shift 2 Unleashed: https://store.steampowered.com/app/47920


In [ ]:
display(desc_df['pc_requirements'].iloc[0])
display(desc_df['pc_requirements'].iloc[2000])
display(desc_df['pc_requirements'].iloc[15000])

"{'minimum': '\\n\\t\\t\\t<p><strong>Minimum:</strong> 500 mhz processor, 96mb ram, 16mb video card, Windows XP, Mouse, Keyboard, Internet Connection<br /></p>\\n\\t\\t\\t<p><strong>Recommended:</strong> 800 mhz processor, 128mb ram, 32mb+ video card, Windows XP, Mouse, Keyboard, Internet Connection<br /></p>\\n\\t\\t\\t'}"

'{\'minimum\': \'<strong>Minimum:</strong><br><ul class="bb_ul"><li><strong>OS *:</strong> Windows XP/Vista/8<br></li><li><strong>Processor:</strong> CPU: 1.0 GHz<br></li><li><strong>Memory:</strong> 512 MB RAM<br></li><li><strong>Graphics:</strong> DirectX: 9.0 Compatible<br></li><li><strong>DirectX:</strong> Version 9.0<br></li><li><strong>Storage:</strong> 531 MB available space<br></li><li><strong>Sound Card:</strong> DirectX: 9.0 Compatible</li></ul>\'}'

'{\'minimum\': \'<strong>Minimum:</strong><br><ul class="bb_ul"><li><strong>OS *:</strong> Windows 7 32-bit or 64-bit<br></li><li><strong>Processor:</strong> i3<br></li><li><strong>Memory:</strong> 2 GB RAM<br></li><li><strong>Graphics:</strong> NVIDIA GeForce 470 GTX or AMD Radeon 6870 HD series card or higher.<br></li><li><strong>DirectX:</strong> Version 11<br></li><li><strong>Storage:</strong> 8 GB available space</li></ul>\', \'recommended\': \'<strong>Recommended:</strong><br><ul class="bb_ul"><li><strong>OS:</strong> Windows 10 32-bit or 64-bit<br></li><li><strong>Processor:</strong> i3<br></li><li><strong>Memory:</strong> 4 GB RAM<br></li><li><strong>Graphics:</strong> NVIDIA GeForce 470 GTX or AMD Radeon 6870 HD series card or higher.<br></li><li><strong>DirectX:</strong> Version 11<br></li><li><strong>Storage:</strong> 8 GB available space</li></ul>\'}'

In [ ]:
view_requirements = desc_df['pc_requirements'].iloc[[0, 2000, 15000]].copy()

view_requirements = (view_requirements
                         .str.replace(r'\\[rtn]', '', regex=True)
                         .str.replace(r'<[pbr]{1,2}>', ' ', regex=True)
                         .str.replace(r'<[\/"=\w\s]+>', '', regex=True)
                    )

for i, row in view_requirements.items():
    display(row)

"{'minimum': ' Minimum: 500 mhz processor, 96mb ram, 16mb video card, Windows XP, Mouse, Keyboard, Internet Connection Recommended: 800 mhz processor, 128mb ram, 32mb+ video card, Windows XP, Mouse, Keyboard, Internet Connection'}"

"{'minimum': 'Minimum: OS *: Windows XP/Vista/8 Processor: CPU: 1.0 GHz Memory: 512 MB RAM Graphics: DirectX: 9.0 Compatible DirectX: Version 9.0 Storage: 531 MB available space Sound Card: DirectX: 9.0 Compatible'}"

"{'minimum': 'Minimum: OS *: Windows 7 32-bit or 64-bit Processor: i3 Memory: 2 GB RAM Graphics: NVIDIA GeForce 470 GTX or AMD Radeon 6870 HD series card or higher. DirectX: Version 11 Storage: 8 GB available space', 'recommended': 'Recommended: OS: Windows 10 32-bit or 64-bit Processor: i3 Memory: 4 GB RAM Graphics: NVIDIA GeForce 470 GTX or AMD Radeon 6870 HD series card or higher. DirectX: Version 11 Storage: 8 GB available space'}"

In [ ]:
def clean_html(text):
    """Removes HTML tags from a given text."""
    return BeautifulSoup(text, "html.parser").get_text(separator=" ").strip()

def process_requirements(df, export=False):
    if export:
        requirements = df[['steamid', 'pc_requirements', 'mac_requirements', 'linux_requirements']].copy()

        # Remove rows where 'pc_requirements' is empty
        requirements = requirements[requirements['pc_requirements'] != '[]']

        # Clean up string formatting issues
        requirements['requirements_clean'] = (requirements['pc_requirements']
                                                  .str.replace(r'\\[rtn]', '', regex=True)
                                                  .str.replace(r'<[pbr]{1,2}>', ' ', regex=True)
                                                  .str.replace(r'<[\/"=\w\s]+>', '', regex=True)
                                             )

        # Convert the string dictionary to actual dictionary
        requirements['requirements_clean'] = requirements['requirements_clean'].apply(lambda x: ast.literal_eval(x))

        # Extract and clean minimum and recommended requirements
        requirements['minimum'] = requirements['requirements_clean'].apply(
            lambda x: clean_html(x['minimum'].replace('Minimum:', '').strip()) if 'minimum' in x else np.nan
        )
        requirements['recommended'] = requirements['requirements_clean'].apply(
            lambda x: clean_html(x['recommended'].replace('Recommended:', '').strip()) if 'recommended' in x else np.nan
        )

        # Drop the intermediate column
        requirements = requirements.drop('requirements_clean', axis=1)

        # Export data if required
        export_data(requirements, 'requirements_data')

    # Remove original columns from df before returning
    df = df.drop(['pc_requirements', 'mac_requirements', 'linux_requirements'], axis=1)

    return df

# Run the function
reqs_df = process_requirements(desc_df, export=True)

Exported requirements data to '/content/drive/MyDrive/Colab Notebooks/steamdata/experimental/datasets_final/steam_requirements_data.csv'


In [ ]:
# verify export
pd.read_csv(f'{dataset_final_dir}/steam_requirements_data.csv').head()

,steamid,pc_requirements,mac_requirements,linux_requirements,minimum,recommended
0,10,{'minimum': '\n\t\t\t<p><strong>Minimum:</stro...,{'minimum': 'Minimum: OS X Snow Leopard 10.6....,"{'minimum': 'Minimum: Linux Ubuntu 12.04, Dual...","500 mhz processor, 96mb ram, 16mb video card, ...",NaN
1,20,{'minimum': '\n\t\t\t<p><strong>Minimum:</stro...,{'minimum': 'Minimum: OS X Snow Leopard 10.6....,"{'minimum': 'Minimum: Linux Ubuntu 12.04, Dual...","500 mhz processor, 96mb ram, 16mb video card, ...",NaN
2,30,{'minimum': '\n\t\t\t<p><strong>Minimum:</stro...,{'minimum': 'Minimum: OS X Snow Leopard 10.6....,"{'minimum': 'Minimum: Linux Ubuntu 12.04, Dual...","500 mhz processor, 96mb ram, 16mb video card, ...",NaN
3,40,{'minimum': '\n\t\t\t<p><strong>Minimum:</stro...,{'minimum': 'Minimum: OS X Snow Leopard 10.6....,"{'minimum': 'Minimum: Linux Ubuntu 12.04, Dual...","500 mhz processor, 96mb ram, 16mb video card, ...",NaN
4,50,{'minimum': '\n\t\t\t<p><strong>Minimum:</stro...,{'minimum': 'Minimum: OS X Snow Leopard 10.6....,"{'minimum': 'Minimum: Linux Ubuntu 12.04, Dual...","500 mhz processor, 96mb ram, 16mb video card, ...",NaN


In [ ]:
# print out number of rows and columns
print('Rows:', reqs_df.shape[0])
print('Columns:', reqs_df.shape[1])

reqs_df.head()

Rows: 84213
Columns: 26


,steamid,id,name_x,price,reviews_x,followers,reviewScore,copiesSold,revenue,wishlists,avgPlaytime,releaseDate,publisherClass,players,owners,required_age,is_free,categories,genres_y,achievements,english,windows,mac,linux,developer,publisher
0,10,10,Counter-Strike,9.99,247199,215044,97,7021860,29305366.0,277200.0,183.686513,973054800000,AAA,11010395,16629587,0,0,Multi-player;PvP;Online PvP;Shared/Split Scree...,Action,0,1,1,1,1,Valve,Valve
1,20,20,Team Fortress Classic,4.99,8607,9617,87,375765,853393.0,37400.0,4.589017,922942800000,AAA,1415339,7823635,0,0,Multi-player;PvP;Online PvP;Shared/Split Scree...,Action,0,1,1,1,1,Valve,Valve
2,30,30,Day of Defeat,4.99,7021,7244,90,230695,495765.0,37400.0,18.550989,1051761600000,AAA,1492839,6938005,0,0,Multi-player;Valve Anti-Cheat enabled;Family S...,Action,0,1,1,1,1,Valve,Valve
3,40,40,Deathmatch Classic,4.99,3136,2723,83,143370,310525.0,14300.0,2.748242,991368000000,AAA,1010152,7236021,0,0,Multi-player;PvP;Online PvP;Shared/Split Scree...,Action,0,1,1,1,1,Valve,Valve
4,50,50,Half-Life: Opposing Force,4.99,24809,10844,95,769170,1571654.0,51700.0,6.825604,941432400000,AAA,2076418,9806968,0,0,Single-player;Multi-player;Valve Anti-Cheat en...,Action,0,1,1,1,1,Gearbox Software,Valve


In [ ]:
# remove the columns id
reqs_df = reqs_df.drop('id', axis=1)

# change the column name name_x to name, reviews_x to reviews, genres_y to genres
reqs_df = reqs_df.rename(columns={'name_x': 'name', 'reviews_x': 'reviews', 'genres_y': 'genres'})

print('Rows:', reqs_df.shape[0])
print('Columns:', reqs_df.shape[1])

reqs_df.head()

Rows: 84213
Columns: 25


,steamid,name,price,reviews,followers,reviewScore,copiesSold,revenue,wishlists,avgPlaytime,releaseDate,publisherClass,players,owners,required_age,is_free,categories,genres,achievements,english,windows,mac,linux,developer,publisher
0,10,Counter-Strike,9.99,247199,215044,97,7021860,29305366.0,277200.0,183.686513,973054800000,AAA,11010395,16629587,0,0,Multi-player;PvP;Online PvP;Shared/Split Scree...,Action,0,1,1,1,1,Valve,Valve
1,20,Team Fortress Classic,4.99,8607,9617,87,375765,853393.0,37400.0,4.589017,922942800000,AAA,1415339,7823635,0,0,Multi-player;PvP;Online PvP;Shared/Split Scree...,Action,0,1,1,1,1,Valve,Valve
2,30,Day of Defeat,4.99,7021,7244,90,230695,495765.0,37400.0,18.550989,1051761600000,AAA,1492839,6938005,0,0,Multi-player;Valve Anti-Cheat enabled;Family S...,Action,0,1,1,1,1,Valve,Valve
3,40,Deathmatch Classic,4.99,3136,2723,83,143370,310525.0,14300.0,2.748242,991368000000,AAA,1010152,7236021,0,0,Multi-player;PvP;Online PvP;Shared/Split Scree...,Action,0,1,1,1,1,Valve,Valve
4,50,Half-Life: Opposing Force,4.99,24809,10844,95,769170,1571654.0,51700.0,6.825604,941432400000,AAA,2076418,9806968,0,0,Single-player;Multi-player;Valve Anti-Cheat en...,Action,0,1,1,1,1,Gearbox Software,Valve


# **Seperating the Caetgories**

In [ ]:
print(reqs_df['categories'][0])
reqs_df['categories'].value_counts().head(10)

Multi-player;PvP;Online PvP;Shared/Split Screen PvP;Valve Anti-Cheat enabled;Family Sharing


,count
categories,
Single-player;Family Sharing,18603
Single-player;Steam Achievements;Family Sharing,8068
Single-player,3528
Single-player;Steam Achievements;Steam Cloud;Family Sharing,3453
Single-player;Steam Achievements;Full controller support;Steam Cloud;Family Sharing,2620
Single-player;Steam Achievements;Full controller support;Family Sharing,2478
Single-player;Full controller support;Family Sharing,2301
Single-player;Partial Controller Support;Family Sharing,2194
Single-player;Steam Achievements;Partial Controller Support;Family Sharing,1494


In [ ]:
# def process_categories(df):
#     """Process supported_languages column into a boolean 'is english' column."""
#     df = df.copy()

#     df["Single-player"] = df["categories"].apply(
#         lambda x: 1 if "Single-player" in str(x) else 0
#     )
#     df = df.drop("categories", axis=1)

#     return df


# steam_dataset = process_categories(reqs_df)
# steam_dataset[["name", "Single-player"]].head()

## New one
def process_categories(df, top_n=20):
    """Process 'categories' column into binary columns for top N most frequent categories."""
    df = df.copy()

    # Convert to strings and split categories
    category_lists = df["categories"].dropna().astype(str).str.split(";|,")  # Handles both ";" and "," separators
    all_categories = [cat.strip() for sublist in category_lists for cat in sublist]

    # Get top N most frequent categories
    from collections import Counter
    top_categories = [cat for cat, _ in Counter(all_categories).most_common(top_n)]

    # Create binary columns
    for cat in top_categories:
        df[cat] = df["categories"].apply(
            lambda x: 1 if pd.notnull(x) and cat in str(x) else 0
        )

    df = df.drop("categories", axis=1)
    return df

steam_dataset = process_categories(reqs_df, top_n=20)
steam_dataset[["name"] + list(steam_dataset.columns[-20:])].head()

,name,Single-player,Family Sharing,Steam Achievements,Steam Cloud,Full controller support,Multi-player,Partial Controller Support,Steam Trading Cards,PvP,Co-op,Steam Leaderboards,Remote Play Together,Online PvP,Shared/Split Screen,Tracked Controller Support,VR Only,Shared/Split Screen PvP,Online Co-op,Stats,Shared/Split Screen Co-op
0,Counter-Strike,0,1,0,0,0,1,0,0,1,0,0,0,1,1,0,0,1,0,0,0
1,Team Fortress Classic,0,1,0,0,0,1,0,0,1,0,0,1,1,1,0,0,1,0,0,0
2,Day of Defeat,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Deathmatch Classic,0,1,0,0,0,1,0,0,1,0,0,1,1,1,0,0,1,0,0,0
4,Half-Life: Opposing Force,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
print('Rows:', steam_dataset.shape[0])
print('Columns:', steam_dataset.shape[1])

steam_dataset.head()

Rows: 84213
Columns: 44


,steamid,name,price,reviews,followers,reviewScore,copiesSold,revenue,wishlists,avgPlaytime,releaseDate,publisherClass,players,owners,required_age,is_free,genres,achievements,english,windows,mac,linux,developer,publisher,Single-player,Family Sharing,Steam Achievements,Steam Cloud,Full controller support,Multi-player,Partial Controller Support,Steam Trading Cards,PvP,Co-op,Steam Leaderboards,Remote Play Together,Online PvP,Shared/Split Screen,Tracked Controller Support,VR Only,Shared/Split Screen PvP,Online Co-op,Stats,Shared/Split Screen Co-op
0,10,Counter-Strike,9.99,247199,215044,97,7021860,29305366.0,277200.0,183.686513,973054800000,AAA,11010395,16629587,0,0,Action,0,1,1,1,1,Valve,Valve,0,1,0,0,0,1,0,0,1,0,0,0,1,1,0,0,1,0,0,0
1,20,Team Fortress Classic,4.99,8607,9617,87,375765,853393.0,37400.0,4.589017,922942800000,AAA,1415339,7823635,0,0,Action,0,1,1,1,1,Valve,Valve,0,1,0,0,0,1,0,0,1,0,0,1,1,1,0,0,1,0,0,0
2,30,Day of Defeat,4.99,7021,7244,90,230695,495765.0,37400.0,18.550989,1051761600000,AAA,1492839,6938005,0,0,Action,0,1,1,1,1,Valve,Valve,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,40,Deathmatch Classic,4.99,3136,2723,83,143370,310525.0,14300.0,2.748242,991368000000,AAA,1010152,7236021,0,0,Action,0,1,1,1,1,Valve,Valve,0,1,0,0,0,1,0,0,1,0,0,1,1,1,0,0,1,0,0,0
4,50,Half-Life: Opposing Force,4.99,24809,10844,95,769170,1571654.0,51700.0,6.825604,941432400000,AAA,2076418,9806968,0,0,Action,0,1,1,1,1,Gearbox Software,Valve,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0


## Adding the Extracted Date Column

In [ ]:
# Convert the extracted date to Unix timestamp
def date_to_unix_timestamp(date_string, format_string):
    """Converts a date string to a Unix timestamp in milliseconds.

    Args:
        date_string: The date string to convert.
        format_string: The format of the date string.

    Returns:
        The Unix timestamp in milliseconds as an integer.
    """
    try:
        date_object = datetime.datetime.strptime(date_string, format_string)
        timestamp_ms = int(time.mktime(date_object.timetuple()) * 1000)
        return timestamp_ms
    except ValueError:
        return None

# Example usage
date_str = "2025-02-20"
format_str = "%Y-%m-%d"
unix_timestamp = date_to_unix_timestamp(date_str, format_str)
print("Unix timestamp (ms):", unix_timestamp)

# Add column
steam_dataset["extract_date"] = unix_timestamp

Unix timestamp (ms): 1740009600000


In [ ]:
print('Rows:', steam_dataset.shape[0])
print('Columns:', steam_dataset.shape[1])

steam_dataset.head()

Rows: 84213
Columns: 45


,steamid,name,price,reviews,followers,reviewScore,copiesSold,revenue,wishlists,avgPlaytime,releaseDate,publisherClass,players,owners,required_age,is_free,genres,achievements,english,windows,mac,linux,developer,publisher,Single-player,Family Sharing,Steam Achievements,Steam Cloud,Full controller support,Multi-player,Partial Controller Support,Steam Trading Cards,PvP,Co-op,Steam Leaderboards,Remote Play Together,Online PvP,Shared/Split Screen,Tracked Controller Support,VR Only,Shared/Split Screen PvP,Online Co-op,Stats,Shared/Split Screen Co-op,extract_date
0,10,Counter-Strike,9.99,247199,215044,97,7021860,29305366.0,277200.0,183.686513,973054800000,AAA,11010395,16629587,0,0,Action,0,1,1,1,1,Valve,Valve,0,1,0,0,0,1,0,0,1,0,0,0,1,1,0,0,1,0,0,0,1740009600000
1,20,Team Fortress Classic,4.99,8607,9617,87,375765,853393.0,37400.0,4.589017,922942800000,AAA,1415339,7823635,0,0,Action,0,1,1,1,1,Valve,Valve,0,1,0,0,0,1,0,0,1,0,0,1,1,1,0,0,1,0,0,0,1740009600000
2,30,Day of Defeat,4.99,7021,7244,90,230695,495765.0,37400.0,18.550989,1051761600000,AAA,1492839,6938005,0,0,Action,0,1,1,1,1,Valve,Valve,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1740009600000
3,40,Deathmatch Classic,4.99,3136,2723,83,143370,310525.0,14300.0,2.748242,991368000000,AAA,1010152,7236021,0,0,Action,0,1,1,1,1,Valve,Valve,0,1,0,0,0,1,0,0,1,0,0,1,1,1,0,0,1,0,0,0,1740009600000
4,50,Half-Life: Opposing Force,4.99,24809,10844,95,769170,1571654.0,51700.0,6.825604,941432400000,AAA,2076418,9806968,0,0,Action,0,1,1,1,1,Gearbox Software,Valve,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1740009600000


## Seperating the Genres

In [ ]:
print(steam_dataset['genres'][0])
steam_dataset['genres'].value_counts().head(10)

Action


,count
genres,
Casual;Indie,5489
Action;Indie,4993
Action;Adventure;Indie,4399
Adventure;Indie,3671
Adventure;Casual;Indie,3088
Action;Casual;Indie,2948
Casual,2861
Indie,2627
Action,2553


In [ ]:
# Step 1: Drop missing values and split genres by ';'
all_genres = steam_dataset['genres'].dropna().apply(lambda x: x.split(';'))

# Step 2: Flatten the list of genre lists into a single list
flat_genres = [genre.strip() for sublist in all_genres for genre in sublist]

# Step 3: Count occurrences
genre_counts = Counter(flat_genres)

# Step 4: Convert to DataFrame and calculate percentage
total_games = len(steam_dataset)
genre_df = (
    pd.DataFrame(genre_counts.items(), columns=['genre', 'count'])
    .sort_values(by='count', ascending=False)
)
genre_df['percentage'] = (genre_df['count'] / total_games * 100).round(2)

# Display result
print(genre_df.head(20))

                    genre  count  percentage
4                   Indie  59698       70.89
6                  Casual  37793       44.88
3               Adventure  34170       40.58
0                  Action  34104       40.50
7              Simulation  16779       19.92
2                Strategy  16075       19.09
5                     RPG  14934       17.73
1            Free To Play   8126        9.65
12                 Sports   3603        4.28
8                  Racing   3062        3.64
9   Massively Multiplayer   1640        1.95
11                Violent    320        0.38
18                   Gore    198        0.24
10                 Nudity     85        0.10
24         Sexual Content     85        0.10
36              Education     21        0.02
51       Game Development     16        0.02
39  Design & Illustration     13        0.02
41   Animation & Modeling     10        0.01
38              Utilities     10        0.01


In [ ]:
def process_genres(df, top_n=10):
    """Process 'genres' column into binary columns for top N most frequent genres."""
    df = df.copy()

    # Convert to strings and split genres
    category_lists = df["genres"].dropna().astype(str).str.split(";|,")  # Handles both ";" and "," separators
    all_genres = [cat.strip() for sublist in category_lists for cat in sublist]

    # Get top N most frequent genres
    from collections import Counter
    top_genres = [cat for cat, _ in Counter(all_genres).most_common(top_n)]

    # Create binary columns
    for cat in top_genres:
        df[cat] = df["genres"].apply(
            lambda x: 1 if pd.notnull(x) and cat in str(x) else 0
        )

    df = df.drop("genres", axis=1)
    return df

steam_dataset = process_genres(steam_dataset, top_n=10)
steam_dataset[["name"] + list(steam_dataset.columns[-10:])].head()

,name,Indie,Casual,Adventure,Action,Simulation,Strategy,RPG,Free To Play,Sports,Racing
0,Counter-Strike,0,0,0,1,0,0,0,0,0,0
1,Team Fortress Classic,0,0,0,1,0,0,0,0,0,0
2,Day of Defeat,0,0,0,1,0,0,0,0,0,0
3,Deathmatch Classic,0,0,0,1,0,0,0,0,0,0
4,Half-Life: Opposing Force,0,0,0,1,0,0,0,0,0,0


In [ ]:
# check for Dota 2 name
steam_dataset[steam_dataset['name'] == 'Dota 2']

,steamid,name,price,reviews,followers,reviewScore,copiesSold,revenue,wishlists,avgPlaytime,releaseDate,publisherClass,players,owners,required_age,is_free,achievements,english,windows,mac,linux,developer,publisher,Single-player,Family Sharing,Steam Achievements,Steam Cloud,Full controller support,Multi-player,Partial Controller Support,Steam Trading Cards,PvP,Co-op,Steam Leaderboards,Remote Play Together,Online PvP,Shared/Split Screen,Tracked Controller Support,VR Only,Shared/Split Screen PvP,Online Co-op,Stats,Shared/Split Screen Co-op,extract_date,Indie,Casual,Adventure,Action,Simulation,Strategy,RPG,Free To Play,Sports,Racing
17,570,Dota 2,0.0,2428646,2353436,82,209335804,2.012824e+09,9900.0,424.067564,1373342400000,AAA,209007762,209007762,0,1,0,1,1,1,1,Valve,Valve,0,0,0,0,0,1,0,1,0,1,0,0,0,0,0,0,0,0,0,0,1740009600000,0,0,0,1,0,1,0,1,0,0


## Developers and Publishers

In [ ]:
steam_dataset[['developer', 'publisher']].iloc[24:28]

,developer,publisher
24,Ritual Entertainment,Ritual Entertainment
25,Ritual Entertainment;Nightdive Studios,Nightdive Studios
26,Introversion Software,Introversion Software
27,Introversion Software,Introversion Software


In [ ]:
print(steam_dataset['developer'][0])
steam_dataset['developer'].value_counts().head(100)

Valve


,count
developer,
EroticGamesClub,201
Choice of Games,168
Laush Dmitriy Sergeevich,149
Boogygames Studios,139
Creobit,137
...,...
Nihon Falcom,29
GentleDriver;Puzzle L Porject,29
Minimol Games,28


In [ ]:
print(steam_dataset['publisher'][0])
steam_dataset['publisher'].value_counts().head(100)

Valve


,count
publisher,
Big Fish Games,497
,352
8floor,268
EroticGamesClub,201
Conglomerate 5,193
...,...
Outright Games Ltd.,50
owlyboi,50
Headup,49


In [ ]:
from sklearn.preprocessing import MultiLabelBinarizer

# Step 1: Split developers and publishers into lists
steam_dataset["developer_list"] = steam_dataset["developer"].fillna("").apply(lambda x: x.split(";"))
steam_dataset["publisher_list"] = steam_dataset["publisher"].fillna("").apply(lambda x: x.split(";"))

# Step 2: Find top 2000 developers and publishers
from collections import Counter

dev_counter = Counter([dev for devs in steam_dataset["developer_list"] for dev in devs])
top_2000_devs = set([dev for dev, _ in dev_counter.most_common(2000)])

pub_counter = Counter([pub for pubs in steam_dataset["publisher_list"] for pub in pubs])
top_2000_pubs = set([pub for pub, _ in pub_counter.most_common(2000)])

# Step 3: Filter lists to only keep top 2000; mark others as 'Other'
def filter_top(items, top_set):
    filtered = [item if item in top_set else "Other" for item in items]
    # Optional: remove duplicates after replacing 'Other'
    return list(set(filtered))

steam_dataset["developer_top"] = steam_dataset["developer_list"].apply(lambda x: filter_top(x, top_2000_devs))
steam_dataset["publisher_top"] = steam_dataset["publisher_list"].apply(lambda x: filter_top(x, top_2000_pubs))

# Step 4: MultiLabelBinarizer encode the filtered columns
mlb_dev = MultiLabelBinarizer()
dev_encoded = mlb_dev.fit_transform(steam_dataset["developer_top"])
dev_df = pd.DataFrame(dev_encoded, columns=[f"dev__{d}" for d in mlb_dev.classes_])

mlb_pub = MultiLabelBinarizer()
pub_encoded = mlb_pub.fit_transform(steam_dataset["publisher_top"])
pub_df = pd.DataFrame(pub_encoded, columns=[f"pub__{p}" for p in mlb_pub.classes_])

# Step 5: Join to main dataset
steam_dataset = pd.concat([steam_dataset, dev_df, pub_df], axis=1)

# Optional: drop old columns
steam_dataset = steam_dataset.drop(columns=["developer", "publisher", "developer_list", "publisher_list", "developer_top", "publisher_top"])


## Clean the Dates

In [ ]:
# Define function to extract year, month, and day
def extract_date_parts_safe(timestamp):
    try:
        if pd.notnull(timestamp) and 0 < timestamp < 4102444800000:  # milliseconds
            dt = datetime.datetime.utcfromtimestamp(timestamp / 1000)  # convert to seconds
            return dt.year, dt.month, dt.day
    except Exception:
        pass
    return None, None, None

# Apply safely to both columns
steam_dataset[['release_year', 'release_month', 'release_day']] = steam_dataset['releaseDate'].apply(
    lambda x: pd.Series(extract_date_parts_safe(x))
)

steam_dataset[['extract_year', 'extract_month', 'extract_day']] = steam_dataset['extract_date'].apply(
    lambda x: pd.Series(extract_date_parts_safe(x))
)

# Drop the original timestamp columns
steam_dataset.drop(['releaseDate', 'extract_date'], axis=1, inplace=True)

# Show the cleaned dataset
print(steam_dataset.head(10))

   steamid                            name  price   reviews  followers  \
0     10.0                  Counter-Strike   9.99  247199.0   215044.0   
1     20.0           Team Fortress Classic   4.99    8607.0     9617.0   
2     30.0                   Day of Defeat   4.99    7021.0     7244.0   
3     40.0              Deathmatch Classic   4.99    3136.0     2723.0   
4     50.0       Half-Life: Opposing Force   4.99   24809.0    10844.0   
5     60.0                        Ricochet   4.99    5917.0     2870.0   
6     70.0                       Half-Life   9.99  144697.0    59939.0   
7     80.0  Counter-Strike: Condition Zero   9.99   26105.0    47115.0   
8    130.0           Half-Life: Blue Shift   4.99   18258.0     8580.0   
9    220.0                     Half-Life 2   9.99  236047.0    93324.0   

   reviewScore  copiesSold     revenue  wishlists  avgPlaytime publisherClass  \
0         97.0   7021860.0  29305366.0   277200.0   183.686513            AAA   
1         87.0    37576

## Handle Categorical Data

In [ ]:
print(steam_dataset['publisherClass'][0])
steam_dataset['publisherClass'].value_counts()

AAA


,count
publisherClass,
Hobbyist,45672
Indie,35006
AA,2665
AAA,870


In [ ]:
# Define the ordinal mapping
publisher_class_map = {
    'Hobbyist': 0,
    'Indie': 1,
    'AA': 2,
    'AAA': 3
}

# Apply the mapping to create a new numeric column
steam_dataset['publisherClass_encoded'] = steam_dataset['publisherClass'].map(publisher_class_map)

# Optional: drop the original if not needed
steam_dataset.drop('publisherClass', axis=1, inplace=True)

# Check the result
print(steam_dataset[['name', 'publisherClass_encoded']].head())

                        name  publisherClass_encoded
0             Counter-Strike                     3.0
1      Team Fortress Classic                     3.0
2              Day of Defeat                     3.0
3         Deathmatch Classic                     3.0
4  Half-Life: Opposing Force                     3.0


## Reorder the Columns

In [ ]:
print('Rows:', steam_dataset.shape[0])
print('Columns:', steam_dataset.shape[1])

Rows: 92877
Columns: 4058


In [ ]:
print(steam_dataset.columns.tolist())

['steamid', 'name', 'price', 'reviews', 'followers', 'reviewScore', 'copiesSold', 'revenue', 'wishlists', 'avgPlaytime', 'players', 'owners', 'required_age', 'is_free', 'achievements', 'english', 'windows', 'mac', 'linux', 'Single-player', 'Family Sharing', 'Steam Achievements', 'Steam Cloud', 'Full controller support', 'Multi-player', 'Partial Controller Support', 'Steam Trading Cards', 'PvP', 'Co-op', 'Steam Leaderboards', 'Remote Play Together', 'Online PvP', 'Shared/Split Screen', 'Tracked Controller Support', 'VR Only', 'Shared/Split Screen PvP', 'Online Co-op', 'Stats', 'Shared/Split Screen Co-op', 'Indie', 'Casual', 'Adventure', 'Action', 'Simulation', 'Strategy', 'RPG', 'Free To Play', 'Sports', 'Racing', 'dev__', 'dev__&y', 'dev__0 Deer Soft', 'dev__07th Expansion', 'dev__100 Cozy Games', 'dev__10tons Ltd', 'dev__11 bit studios', 'dev__14Dimension Enterprise', 'dev__16air', 'dev__17Studio', 'dev__1C:InoCo', 'dev__2 Barrel Arts', 'dev__2024-NGame', 'dev__2K Australia', 'dev__2e

In [ ]:
cols_to_move = [
    'players',
    'owners',
    'wishlists',
    'copiesSold',
    'revenue',
    'avgPlaytime',
    'followers',
    'reviews',
    'reviewScore',
]

# Get current columns
cols = steam_dataset.columns.tolist()

# Remove the columns to move from current columns list
for c in cols_to_move:
    if c in cols:
        cols.remove(c)

# Append them at the end
new_order = cols + cols_to_move

# Reorder the dataframe
steam_dataset = steam_dataset[new_order]

In [ ]:
cols_to_front = [
    'steamid',
    'name',
    'price',
    'is_free',
    'release_year',
    'release_month',
    'release_day',
    'extract_year',
    'extract_month',
    'extract_day',
    'publisherClass_encoded',
    'required_age',
    'achievements',
    'english',
    'windows',
    'mac',
    'linux',
]

cols = steam_dataset.columns.tolist()

# Remove these columns from the current list
for c in cols_to_front:
    if c in cols:
        cols.remove(c)

# New order: these front columns first, then the rest
new_order = cols_to_front + cols

steam_dataset = steam_dataset[new_order]


In [ ]:
print('Rows:', steam_dataset.shape[0])
print('Columns:', steam_dataset.shape[1])

Rows: 92877
Columns: 4058


In [ ]:
steam_dataset.head(10)

,steamid,name,price,is_free,release_year,release_month,release_day,extract_year,extract_month,extract_day,publisherClass_encoded,required_age,achievements,english,windows,mac,linux,Single-player,Family Sharing,Steam Achievements,Steam Cloud,Full controller support,Multi-player,Partial Controller Support,Steam Trading Cards,PvP,Co-op,Steam Leaderboards,Remote Play Together,Online PvP,Shared/Split Screen,Tracked Controller Support,VR Only,Shared/Split Screen PvP,Online Co-op,Stats,Shared/Split Screen Co-op,Indie,Casual,Adventure,Action,Simulation,Strategy,RPG,Free To Play,Sports,Racing,dev__,dev__&y,dev__0 Deer Soft,...,pub__randomcreations,pub__resnijars,pub__rokaplay,pub__rondomedia GmbH,pub__russpuppy,pub__samuel crevier,pub__simplecakestudio,pub__socah,pub__studio wasp,pub__theNeko,pub__tinyBuild,pub__tjern,pub__upjers,pub__wang huimin,pub__wow wow Games,pub__xFAIRx,pub__yokaze,pub__zzzgame,pub__ЛД: Новости визуальных новелл,pub__かべろくん,pub__やぶから堂,pub__七彩绘色游戏,pub__东游鉴,pub__个人,pub__冷笑黑妖,pub__凝冰剑斩,pub__吃了就睡工作室,pub__哈视奇科技,pub__大きくて強い,pub__方块游戏(CubeGame),pub__杰游科技,pub__橙光游戏,pub__熊窝dodox,pub__电钮组,pub__白信鸽工作室,pub__致意,pub__臻新,pub__艾露比工作室,pub__迷糊的安安,pub__青灯独酌,pub__유진게임즈,players,owners,wishlists,copiesSold,revenue,avgPlaytime,followers,reviews,reviewScore
0,10.0,Counter-Strike,9.99,0.0,2000.0,11.0,1.0,2025.0,2.0,20.0,3.0,0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,11010395.0,16629587.0,277200.0,7021860.0,29305366.0,183.686513,215044.0,247199.0,97.0
1,20.0,Team Fortress Classic,4.99,0.0,1999.0,4.0,1.0,2025.0,2.0,20.0,3.0,0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1415339.0,7823635.0,37400.0,375765.0,853393.0,4.589017,9617.0,8607.0,87.0
2,30.0,Day of Defeat,4.99,0.0,2003.0,5.0,1.0,2025.0,2.0,20.0,3.0,0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1492839.0,6938005.0,37400.0,230695.0,495765.0,18.550989,7244.0,7021.0,90.0
3,40.0,Deathmatch Classic,4.99,0.0,2001.0,6.0,1.0,2025.0,2.0,20.0,3.0,0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1010152.0,7236021.0,14300.0,143370.0,310525.0,2.748242,2723.0,3136.0,83.0
4,50.0,Half-Life: Opposing Force,4.99,0.0,1999.0,11.0,1.0,2025.0,2.0,20.0,3.0,0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2076418.0,9806968.0,51700.0,769170.0,1571654.0,6.825604,10844.0,24809.0,95.0
5,60.0,Ricochet,4.99,0.0,2000.0,11.0,1.0,2025.0,2.0,20.0,3.0,0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,

In [ ]:
steam_dataset.isnull().sum()

,0
steamid,8664
name,8664
price,8664
is_free,8664
release_year,9009
...,...
revenue,8664
avgPlaytime,8664
followers,8664
reviews,8664


In [ ]:
steam_dataset_cleaned = steam_dataset.dropna()

In [ ]:
print(steam_dataset_cleaned.shape)
print(steam_dataset_cleaned.isnull().sum().sum())

(75460, 4058)
0


In [ ]:
steam_dataset.head(10)

,steamid,name,price,is_free,release_year,release_month,release_day,extract_year,extract_month,extract_day,publisherClass_encoded,required_age,achievements,english,windows,mac,linux,Single-player,Family Sharing,Steam Achievements,Steam Cloud,Full controller support,Multi-player,Partial Controller Support,Steam Trading Cards,PvP,Co-op,Steam Leaderboards,Remote Play Together,Online PvP,Shared/Split Screen,Tracked Controller Support,VR Only,Shared/Split Screen PvP,Online Co-op,Stats,Shared/Split Screen Co-op,Indie,Casual,Adventure,Action,Simulation,Strategy,RPG,Free To Play,Sports,Racing,dev__,dev__&y,dev__0 Deer Soft,...,pub__randomcreations,pub__resnijars,pub__rokaplay,pub__rondomedia GmbH,pub__russpuppy,pub__samuel crevier,pub__simplecakestudio,pub__socah,pub__studio wasp,pub__theNeko,pub__tinyBuild,pub__tjern,pub__upjers,pub__wang huimin,pub__wow wow Games,pub__xFAIRx,pub__yokaze,pub__zzzgame,pub__ЛД: Новости визуальных новелл,pub__かべろくん,pub__やぶから堂,pub__七彩绘色游戏,pub__东游鉴,pub__个人,pub__冷笑黑妖,pub__凝冰剑斩,pub__吃了就睡工作室,pub__哈视奇科技,pub__大きくて強い,pub__方块游戏(CubeGame),pub__杰游科技,pub__橙光游戏,pub__熊窝dodox,pub__电钮组,pub__白信鸽工作室,pub__致意,pub__臻新,pub__艾露比工作室,pub__迷糊的安安,pub__青灯独酌,pub__유진게임즈,players,owners,wishlists,copiesSold,revenue,avgPlaytime,followers,reviews,reviewScore
0,10.0,Counter-Strike,9.99,0.0,2000.0,11.0,1.0,2025.0,2.0,20.0,3.0,0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,11010395.0,16629587.0,277200.0,7021860.0,29305366.0,183.686513,215044.0,247199.0,97.0
1,20.0,Team Fortress Classic,4.99,0.0,1999.0,4.0,1.0,2025.0,2.0,20.0,3.0,0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1415339.0,7823635.0,37400.0,375765.0,853393.0,4.589017,9617.0,8607.0,87.0
2,30.0,Day of Defeat,4.99,0.0,2003.0,5.0,1.0,2025.0,2.0,20.0,3.0,0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1492839.0,6938005.0,37400.0,230695.0,495765.0,18.550989,7244.0,7021.0,90.0
3,40.0,Deathmatch Classic,4.99,0.0,2001.0,6.0,1.0,2025.0,2.0,20.0,3.0,0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1010152.0,7236021.0,14300.0,143370.0,310525.0,2.748242,2723.0,3136.0,83.0
4,50.0,Half-Life: Opposing Force,4.99,0.0,1999.0,11.0,1.0,2025.0,2.0,20.0,3.0,0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2076418.0,9806968.0,51700.0,769170.0,1571654.0,6.825604,10844.0,24809.0,95.0
5,60.0,Ricochet,4.99,0.0,2000.0,11.0,1.0,2025.0,2.0,20.0,3.0,0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,

In [ ]:
print(steam_dataset['release_year'][0])
steam_dataset['release_year'].value_counts()

2000.0


,count
release_year,
2024.0,16262
2023.0,11891
2022.0,10139
2021.0,9321
2020.0,7843
2018.0,6494
2019.0,6369
2017.0,5134
2016.0,3635


In [ ]:
# Remove rows where release_year is 2030
steam_dataset = steam_dataset[steam_dataset['release_year'] != 2030.0]

# Optional: reset index after removal
steam_dataset.reset_index(drop=True, inplace=True)

# Confirm removal
print(steam_dataset['release_year'].value_counts().sort_index(ascending=False))

release_year
2025.0     1260
2024.0    16262
2023.0    11891
2022.0    10139
2021.0     9321
2020.0     7843
2019.0     6369
2018.0     6494
2017.0     5134
2016.0     3635
2015.0     2276
2014.0     1417
2013.0      435
2012.0      311
2011.0      248
2010.0      237
2009.0      306
2008.0      137
2007.0       79
2006.0       57
2005.0        2
2004.0        2
2003.0        2
2002.0        1
2001.0        2
2000.0        2
1999.0        2
1998.0        1
1997.0        2
Name: count, dtype: int64


In [ ]:
# row_dict = steam_dataset.iloc['45'].to_dict()
# print(row_dict)

In [ ]:
# elden_ring_rows = steam_dataset[steam_dataset['name'].str.lower() == 'elden ring'.lower()]
# print(elden_ring_rows.to_dict())

In [ ]:
steam_dataset.to_csv('/content/drive/MyDrive/Colab Notebooks/steamdata/experimental/datasets_final/steam_dataset_2000_dev_pub.csv', index=False)

In [ ]:
# # # Sort by year, then month, then day in descending order
# # recent_games = steam_dataset.sort_values(
# #     by=['release_year', 'release_month', 'release_day'],
# #     ascending=[False, False, False]
# # )

# # # Select the most recent 500 games
# # most_recent_games = recent_games.head(500)

# # # Define the output path for the new CSV file
# # output_path = '/content/drive/MyDrive/Colab Notebooks/steamdata/experimental/datasets_final/recent_500_games_3.csv'

# # # Save this subset to a new CSV file
# # most_recent_games.to_csv(output_path, index=False)

# # print(f"Saved the most recent 500 games to: {output_path}")

# # Filter games released in 2023
# games_2023 = steam_dataset[steam_dataset['release_year'] == 2023.0]

# # Sort by release_month and release_day in descending order to get the most recent ones
# games_2023_sorted = games_2023.sort_values(
#     by=['release_month', 'release_day'],
#     ascending=[False, False]
# )

# # Select the top 500 most recent games of 2023
# top_500_2023_games = games_2023_sorted.head(500)

# # Define the output path
# output_path_2023 = '/content/drive/MyDrive/Colab Notebooks/steamdata/experimental/datasets_final/recent_500_games_2023.csv'

# # Export to CSV
# top_500_2023_games.to_csv(output_path_2023, index=False)

# print(f"Saved the most recent 500 games from 2023 to: {output_path_2023}")

In [ ]:
# # --- 1. Calculate Unix millisecond timestamps for 2020 and 2023 ---

# # # Start of 2020 (January 1, 2020, 00:00:00 UTC)
# # start_2020 = datetime.datetime(2020, 1, 1, 0, 0, 0, tzinfo=datetime.timezone.utc)
# # unix_start_2020_ms = int(start_2020.timestamp() * 1000)

# # # End of 2023 (December 31, 2023, 23:59:59 UTC)
# # # We use 23:59:59 to ensure all of 2023 is included.
# # end_2023 = datetime.datetime(2023, 12, 31, 23, 59, 59, 999999, tzinfo=datetime.timezone.utc)
# # unix_end_2023_ms = int(end_2023.timestamp() * 1000)

# # print(f"Unix timestamp for start of 2020 (ms): {unix_start_2020_ms}")
# # print(f"Unix timestamp for end of 2023 (ms): {unix_end_2023_ms}")


# # --- 2. Filter the DataFrame using the Unix timestamps ---

# games_from_to = steam_dataset[
#     (steam_dataset['release_year'] >= 2020) &
#     (steam_dataset['release_year'] <= 2023)
# ]

# # --- 3. Save the filtered DataFrame to a new CSV file ---

# output_path_range = '/content/drive/MyDrive/Colab Notebooks/steamdata/experimental/datasets_final/games_2020_to_2023_5.csv'
# games_from_to.to_csv(output_path_range, index=False)

# print(f"Successfully saved games from 2020 to 2023 to: {output_path_range}")